In [ ]:
!pip install ../input/hpacellsegmentatormaster/HPA-Cell-Segmentation-master/
!pip install ../input/hpapytorchzoozip/pytorch_zoo-master/
!pip install ../input/mmdetection-v280/src/mmpycocotools-12.0.3/mmpycocotools-12.0.3/


In [ ]:
import numpy as np
import pandas as pd

import os
import sys
import cv2 
import math
from tqdm import tqdm
import pickle
from itertools import groupby
import matplotlib.pyplot as plt
import os
import base64
import typing as t
import zlib
import networkx as nx
import random
from timeit import default_timer as timer
random.seed(0)
from tqdm import tqdm

PRINTINFO=False
SHOWIMAGES=False

from pycocotools import mask as mutils
from pycocotools import _mask as coco_mask

from skimage import feature
from joblib import dump, load

exp_name = "v1"
conf_name = "Dataministic-with-CellSeg"
model_name = 'CellSegOpenCV'
ROOT = '../input/hpa-single-cell-image-classification/'

###### Change here between test and train

train_or_test = 'test'

######

debug = False
debug_show_nucleolus_masks=False
debug_show_input_images=False
if train_or_test == 'test':
    data_df = pd.read_csv(os.path.join(ROOT, 'sample_submission.csv'))

    if len(data_df) == 559:
        debug = True
        #one_frame = [data_df.loc[(data_df['ID']== '0173029a-161d-40ef-af28-2342915b22fb') |\
        #             (data_df['ID']=='00c9a1c9-2f06-476f-8b0d-6d01032874a2')]]

        data_df = data_df[:3]
        #data_df = pd.concat(one_frame)
        #print(data_df)
    else:
        debug = False
    
else:
    debug = True
    train_df = pd.read_csv(os.path.join(ROOT, 'train.csv'))

    if train_or_test == 'traintest':
        of_each = 3
        tdf0 = train_df.loc[train_df['Label'] == '0'].tail(of_each)
        tdf1 = train_df.loc[train_df['Label'] == '1'].tail(of_each)
        tdf2 = train_df.loc[train_df['Label'] == '2'].tail(of_each)
        tdf3 = train_df.loc[train_df['Label'] == '3'].tail(of_each)
        tdf4 = train_df.loc[train_df['Label'] == '4'].tail(of_each)
        tdf5 = train_df.loc[train_df['Label'] == '5'].tail(of_each)
        tdf6 = train_df.loc[train_df['Label'] == '6'].tail(of_each)
        tdf7 = train_df.loc[train_df['Label'] == '7'].tail(of_each)
        tdf8 = train_df.loc[train_df['Label'] == '8'].tail(of_each)
        tdf9 = train_df.loc[train_df['Label'] == '9'].tail(of_each)
        tdf10 = train_df.loc[train_df['Label'] == '10'].tail(of_each)
        tdf11 = train_df.loc[train_df['Label'] == '11'].tail(of_each)
        tdf12 = train_df.loc[train_df['Label'] == '12'].tail(of_each)
        tdf13 = train_df.loc[train_df['Label'] == '13'].tail(of_each)
        tdf14 = train_df.loc[train_df['Label'] == '14'].tail(of_each)
        tdf15 = train_df.loc[train_df['Label'] == '15'].tail(of_each)
        tdf16 = train_df.loc[train_df['Label'] == '16'].tail(of_each)
        tdf17 = train_df.loc[train_df['Label'] == '17'].tail(of_each)
        tdf18 = train_df.loc[train_df['Label'] == '18'].tail(of_each)
    else:
        of_each = 20
        tdf0 = train_df.loc[train_df['Label'] == '0'].head(of_each)
        tdf1 = train_df.loc[train_df['Label'] == '1'].head(of_each)
        tdf2 = train_df.loc[train_df['Label'] == '2'].head(of_each)
        tdf3 = train_df.loc[train_df['Label'] == '3'].head(of_each)
        tdf4 = train_df.loc[train_df['Label'] == '4'].head(of_each)
        tdf5 = train_df.loc[train_df['Label'] == '5'].head(of_each)
        tdf6 = train_df.loc[train_df['Label'] == '6'].head(of_each)
        tdf7 = train_df.loc[train_df['Label'] == '7'].head(of_each)
        tdf8 = train_df.loc[train_df['Label'] == '8'].head(of_each)
        tdf9 = train_df.loc[train_df['Label'] == '9'].head(of_each)
        tdf10 = train_df.loc[train_df['Label'] == '10'].head(of_each)
        tdf11 = train_df.loc[train_df['Label'] == '11'].head(of_each)
        tdf12 = train_df.loc[train_df['Label'] == '12'].head(of_each)
        tdf13 = train_df.loc[train_df['Label'] == '13'].head(of_each)
        tdf14 = train_df.loc[train_df['Label'] == '14'].head(of_each)
        tdf15 = train_df.loc[train_df['Label'] == '15'].head(of_each)
        tdf16 = train_df.loc[train_df['Label'] == '16'].head(of_each)
        tdf17 = train_df.loc[train_df['Label'] == '17'].head(of_each)
        tdf18 = train_df.loc[train_df['Label'] == '18'].head(of_each)    
        
    frames=[
            tdf0,
            tdf1,
            tdf2,
            tdf3,
            tdf4,
            tdf5,
            tdf6,
            tdf7,
            tdf8,
            tdf9,
            tdf10,
            tdf11,
            tdf12,
            tdf13,
            tdf14,
            tdf15,
            tdf16,
            tdf17,
            tdf18        
        ]
    
        
    first_frame = [tdf16]
     
    #one_frame = [train_df.loc[train_df['ID']== '49b573a7-a887-499e-b96b-28fbe3fae7f7'].head(1)]
    one_frame = [train_df.loc[train_df['ID']==  '4870f3ee-bb9b-11e8-b2b9-ac1f6b6435d0']]

    #data_df = pd.concat(frames)
    data_df = pd.concat(first_frame)
    #data_df = pd.concat(one_frame)
    
    #tdf0nuc = analyzeNucleusImages(tdf0, train_or_test)
    #tdf1nuc = analyzeNucleusImages(tdf1, train_or_test)
    #tdf2nuc = analyzeNucleusImages(tdf2, train_or_test)
    #tdf3nuc = analyzeNucleusImages(tdf3, train_or_test)
    #tdf4nuc = analyzeNucleusImages(tdf4, train_or_test)
    #tdf5nuc = analyzeNucleusImages(tdf5, train_or_test) 
print(data_df)    
print(debug)

In [ ]:
def encode_binary_mask(mask: np.ndarray) -> t.Text:
  """Converts a binary mask into OID challenge encoding ascii text."""

  # check input mask --
  if mask.dtype != np.bool:
    raise ValueError(
        "encode_binary_mask expects a binary mask, received dtype == %s" %
        mask.dtype)

  mask = np.squeeze(mask)
  if len(mask.shape) != 2:
    raise ValueError(
        "encode_binary_mask expects a 2d mask, received shape == %s" %
        mask.shape)

  # convert input mask to expected COCO API input --
  mask_to_encode = mask.reshape(mask.shape[0], mask.shape[1], 1)
  mask_to_encode = mask_to_encode.astype(np.uint8)
  mask_to_encode = np.asfortranarray(mask_to_encode)

  # RLE encode mask --
  encoded_mask = coco_mask.encode(mask_to_encode)[0]["counts"]

  # compress and base64 encoding --
  binary_str = zlib.compress(encoded_mask, zlib.Z_BEST_COMPRESSION)
  base64_str = base64.b64encode(binary_str)
  return base64_str.decode()

In [ ]:
!ls ../input/hpa-single-cell-image-classification/test/49b573a7-a887-499e-b96b-28fbe3fae7f7* 


In [ ]:
def printinfo (infostring):
    if PRINTINFO:
        print(infostring)

In [ ]:
def find_nuc_contours (nu):
    
    nugauss = cv2.GaussianBlur(nu, (5, 5), 0)
    nuc_thresh = cv2.adaptiveThreshold(nugauss,255,cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY,201,0)
    
    
    # Find the contours on the nuclei image
    contours, hierarchy = cv2.findContours(nuc_thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    # Disregard any contours that are too small to be a nucleus
    i=0
    num_nuclei=0
    nuc_cont=[]
    nuc_midpoint=[]
    nuc_rect=[]
    nuc_mask = np.zeros(nu.shape, dtype='uint8')
    
    for x in contours:
        contlength = cv2.arcLength(contours[i],True)
        nucarea = cv2.contourArea(contours[i])
        approx = cv2.approxPolyDP(x,5,True)
        hull = cv2.convexHull(approx)
        hullArcLength = cv2.arcLength(hull,True)


        printinfo(f'nucleus {num_nuclei} has arc length {contlength}, hullArcLength {hullArcLength},area {nucarea} ')
        
        if ( hullArcLength > 300 and hierarchy[0][i][3] == -1 ):
            nuc_cont.append(hull)
            cv2.drawContours(nuc_mask, nuc_cont,num_nuclei,num_nuclei+1,cv2.FILLED,cv2.LINE_8)
            num_nuclei+=1
            
            M = cv2.moments(contours[i])
            cx = int(M['m10']/M['m00'])
            cy = int(M['m01']/M['m00'])
            nuc_midpoint.append((cx,cy))
              
            rect = cv2.boundingRect(contours[i])
            nuc_rect.append(rect)

            #printinfo('Chosen')
        i=i+1
    #display_image_big(nu)
    #display_image_big(nuc_mask)
    return nuc_cont, nuc_midpoint, nuc_mask, nuc_rect

In [ ]:
def show_mt_er_nu(mt, er, nu):
   
    fig, ax = plt.subplots(1,4, figsize=(20,50))
    ax[0].imshow(mt)
    ax[0].axis('off')

    ax[1].imshow(er)
    ax[1].axis('off')

    ax[2].imshow(nu)
    ax[2].axis('off')

    ax[3].imshow(er)
    ax[3].axis('off')

    plt.show()

In [ ]:
def generateLayoutGraph(image_data, nuc_cont, nuc_midpoint, nuc_rect):
    cell_graph = nx.Graph()

    for j in range(len(nuc_cont)):
        cell_graph.add_node(j)
        
    return cell_graph

In [ ]:
def imageLevelStats( img, image_data):
    maxval=img.max()
    minval=img.min()
    sumval=img.sum()
    hist_im = np.histogram(img, bins=[0,31,63,95,127,159,191,223,255])
    hist256=np.histogram(img, bins=256)

    maxfreq=0
    for i in range(256):
        if hist256[0][i]>maxfreq:
            maxfreq=hist256[0][i]
            mostfreq=i
    medianval = 0
     
    return maxval, minval, sumval, maxfreq, mostfreq, medianval



In [ ]:
def choose_threshold(img, max_pixels_above):
    hist256=np.histogram(img, bins=256)

    numpix=0
    for i in range(255,-1,-1):
        numpix = numpix + hist256[0][i]
        if numpix > max_pixels_above:
            break
    return i

In [ ]:
def find_cells( mt, er, nu, nuc_cont, nuc_mask, nuc_midpoint, cell_graph):
    BOXWIDTH = 5
    RAYLENGTH=500
    BLACK_THRESHOLD=10
    BLUE_THRESHOLD=25
    MIN_BOX_AREA=25
    ryb = cv2.merge([mt, er, nu])

    #displaycopy= np.copy(ryb)
    #cv2.drawContours(displaycopy,nuc_cont,-1,(200,200,255),9)
    #do_display_image_big(displaycopy)
    image_new_contours=[]
    imagesize=np.array(nu.shape)

    #print('imaggesize', imagesize)

    for j in range(len(nuc_cont)):
        printinfo(f'Contour {j}')
        cont = nuc_cont[j]
        #print(nuc_cont[j].shape)
        printinfo(f'midpoint {nuc_midpoint[j]}')
        mid = np.array(nuc_midpoint[j])
        #print('mid',mid)
        #print(type(nuc_midpoint[j]))
        #for k in range(len(cont)-1):
        newcont=[]

        
        for k in range(len(cont)):
            printinfo(f'Contour {j} Segment {k}')
            edge_start= nuc_cont[j][k][0]
            if (k == len(cont)-1):
                # end of the last segment is the start point
                edge_end= nuc_cont[j][0][0]
            else:
                edge_end= nuc_cont[j][k+1][0]

            adjustedEdgeStart=edge_start 
            adjustedEdgeEnd=edge_end        
            printinfo(f'edge_start {edge_start} edge_end {edge_end}')
            edgevec = edge_end - edge_start
            if (edgevec[0]**2+edgevec[1]**2 < 25):
                printinfo('Warning: Very small contour segment')
            #print('edgevec',edgevec)
            midvec = edge_start + 0.5*edgevec
            #print('midvec',midvec)
            perpvec = midvec - mid
            #print(perpvec)
            scalefac=1/abs(perpvec).sum()
            perpvecW=perpvec* BOXWIDTH *scalefac
            perprayvec=perpvec*RAYLENGTH*scalefac
            #print('perpvecs')
            #print(perpvecW)
            #print(perprayvec)
            boxContour=np.array([edge_start,edge_end,edge_end,edge_start])

            #boxContour=np.array([nuc_cont[j][k][0],nuc_cont[j][k+1][0],nuc_cont[j][k+1][0],nuc_cont[j][k][0]])
            #rayBox=np.array([nuc_cont[j][k][0],nuc_cont[j][k+1][0],nuc_cont[j][k+1][0],nuc_cont[j][k][0]])

            #print('boxContour')
            #print(boxContour)
            #print(boxContour.shape)
            boxContour[2]=boxContour[2]+perpvecW
            boxContour[3]=boxContour[3]+perpvecW
            # Keep the floating values in boxContourF so that small increments don't get lost in rounding to pixels
            boxContourF=boxContour
            # Continue with rounded values in boxContour
            boxContour=boxContourF.astype(int)
        
            #rayBox[2]=rayBox[2]+perprayvec
            #rayBox[3]=rayBox[3]+perprayvec
            #print('rayBox')
            #print(rayBox)
            #clippedRayBox=np.fmin(rayBox,imagesize)
            #print('clippedRayBox')
            #print(clippedRayBox)

            bluefound = False
            blackfound = False
            boxContourAreaInitial=cv2.contourArea(boxContour)
            #print('boxContourAreaInitial', boxContourAreaInitial)
            if boxContourAreaInitial < MIN_BOX_AREA:
                printinfo('Error/ToDo remove small contour edges')      
            
            for n in range(1,RAYLENGTH//BOXWIDTH):
                boxContourClipLow = np.fmax(boxContour,[0,0]) 
                boxContourClipped = np.fmin(boxContourClipLow,imagesize)
                boxContourArea=cv2.contourArea(boxContourClipped)
                #print('boxContourArea', boxContourArea)
                    
                if boxContourClipped[2][0] < boxContour[2][0] or boxContourClipped[3][0] < boxContour[3][0]:
                    printinfo('hit right image boundary')
                if boxContourClipped[2][1] < boxContour[2][1] or boxContourClipped[3][1] < boxContour[3][1]:
                    printinfo('hit bottom image boundary')
                if boxContourClipped[2][0] > boxContour[2][0] or boxContourClipped[3][0] > boxContour[3][0]:
                    printinfo('hit left image boundary')
                if boxContourClipped[2][1] > boxContour[2][1] or boxContourClipped[3][1] > boxContour[3][1]:
                    printinfo('hit top image boundary') 

                if bluefound or blackfound or boxContourArea < MIN_BOX_AREA:
                    break
                # boxContour is now clipped so that it remains in the image
                boxContour=boxContourClipped
                boxContourF=np.copy(boxContourClipped)                           
                #if j==0 and k == 3 :
                    #print('boxContour')
                    #print(boxContour) 
                adjustedEdgeStart=boxContour[0]
                adjustedEdgeEnd=boxContour[1]
                x,y,w,h = cv2.boundingRect(boxContour)
                #print(x,y,w,h)
                enclosingSlice= np.copy(ryb[y:y+h, x:x+w])
                # sliceBoxContour is the contour translated from image coordinates to 
                # a slice of the image containing the bounding box of the contour
                sliceBoxContour= boxContour - [x,y]
                #print('sliceBoxContour Shape',sliceBoxContour.shape )
                #print(sliceBoxContour)
                boxMask = np.zeros(enclosingSlice.shape, dtype='uint8')
                cv2.drawContours(boxMask,[sliceBoxContour],-1,(255,255,255),cv2.FILLED,cv2.LINE_8)
                numMaskPixels = np.sum(boxMask==255)/3
                #print('enclosingSlice',enclosingSlice.shape)
                enclosingSlice[boxMask==(0,0,0)]=0
                #do_display_image_big(boxMask)
                #do_display_image_big(enclosingSlice)
                
                #print('numMaskPixels',numMaskPixels)
                #print('boxMask.shape',boxMask.shape)
                #print('enclosingSlice.shape',enclosingSlice.shape)
                #p.set_printoptions(threshold=200)
                #np.set_printoptions(threshold=sys.maxsize)
            
                cropped_mt_sum, cropped_er_sum, cropped_nu_sum = np.sum(enclosingSlice, axis=(0,1) )
                
                #print('cropped_mt_sum',cropped_mt_sum)
                #print('cropped_er_sum',cropped_er_sum)
                #print('cropped_nu_sum',cropped_nu_sum)
                
                if numMaskPixels > 0:
                    mt_erMean = (cropped_mt_sum+cropped_er_sum)/numMaskPixels
                    nuMean = cropped_nu_sum/numMaskPixels
                    #if j==1 and k == 5 :
                        #print('numMaskPixels',numMaskPixels)
                        #print('cropped_mt_sum',cropped_mt_sum)
                        #print('cropped_er_sum',cropped_er_sum)
                        #print('cropped_nu_sum',cropped_nu_sum)
                        #print(f'nuMean:{nuMean} mt_erMean:{mt_erMean}')
                    if nuMean > BLUE_THRESHOLD:

                            # Take a slice from the nucleus mask corresponding to the 
                            # current box where blue was found
                            # Look at the nucleus mask values to see which nucleus we hit
                            # sum the non-zero values and find the mean
                            # 
                            nuc_maskSlice= np.copy(nuc_mask[y:y+h, x:x+w])
                            boxMask0 = boxMask[:,:,1]
                            nuc_maskSlice[boxMask0==0]=0
                            contours_hit=[]
                            cont1=np.amax(nuc_maskSlice)
                            if cont1 != j+1 and cont1 != 0 and cont1 != 255:
                                contours_hit.append(cont1)
                                cell_graph.add_edge(j, cont1-1)
                            nuc_maskSlice[nuc_maskSlice==0]=255
                            cont2=np.amin(nuc_maskSlice)
                            if (cont2 != j+1) and (cont2 != cont1) and cont1 != 0 and cont1 != 255:
                                contours_hit.append(cont2)
                                cell_graph.add_edge(j, cont2-1)
                            if len(contours_hit)>0:
                                bluefound=True
                                if len(contours_hit)>1:
                                    printinfo("Error/Todo: hit two nuclei")
                                printinfo(f'hit contours {contours_hit}')

                    else:
                        if mt_erMean < BLACK_THRESHOLD:
                            blackfound=True
                else:
                    printinfo('!!! Error: unexpected empty mask !!!')
                boxContourF=boxContourF+perpvecW
                boxContour=boxContourF.astype(int)
            #print(f'Terminated after {n} iterations')
            if bluefound:
                printinfo('Blue found')
                printinfo(f'edge_start:{edge_end}')
                printinfo(f'adjustedEdgeStart: {adjustedEdgeEnd}')
                newstart=(adjustedEdgeStart+edge_start)/2
                newend=(adjustedEdgeEnd+edge_end)/2
                adjustedEdgeStart = newstart.astype(int)
                adjustedEdgeEnd = newend.astype(int)
                printinfo('f {newstart} {newend}')
            if blackfound:
                printinfo('Black found')
            if boxContourArea < MIN_BOX_AREA:
                printinfo('Hit edge of image')
            printinfo(f'Adjusted edge is {adjustedEdgeStart} , {adjustedEdgeEnd}')
            newcont.append(adjustedEdgeStart)
            newcont.append(adjustedEdgeEnd)
        
        cnt=np.array(newcont)
        image_new_contours.append(cnt)
        #print('cnt.shape', cnt.shape )
        #print(cnt)
        #print('cnt.type', cnt.dtype )
        cv2.drawContours(ryb,[cnt],-1,(200,200,(255-10*(min(j,20)))),3)


    #print('ryb.shape', ryb.shape )
    #cv2.drawContours(ryb,[cnt],-1,(200,200,255),9) 
    #do_display_image_big(ryb)
    
    return image_new_contours

In [ ]:
def do_display_image_big(im_data, image_data=None):

    dpi = 62
     
    height = im_data.shape[0]
    width = im_data.shape[1]

    # What size does the figure need to be in inches to fit the image?
    figsize = width / float(dpi), height / float(dpi)

    # Create a figure of the right size with one axes that takes up the full figure
    fig = plt.figure(figsize=figsize)
    fig.set_dpi(dpi)
    ax = fig.add_axes([0, 0, 1, 1])

    # Hide spines, ticks, etc.
    ax.axis('off')

    # Display the image.
    ax.imshow(im_data)
    
    if not image_data == None:
        for cell in image_data['cells']:
            cellnum = cell['cellnum']
            x,y,w,h = cell['nuc_bbox']
            ax.text(x,y,cellnum , fontsize=80)

    plt.show()
    
def display_image_big(im_data):
    if SHOWIMAGES:
        do_display_image_big(im_data)

In [ ]:
def newCellDict(image_id, image_label, x, y, width, height  ):
    cell= {
            'image_id': image_id,
            'image_label': image_label,
            'cell_results': [],
            'cellnum': None,
            'bbox': (x,y,width,height),
            'nuc_bbox': None,
            'nuc_midpoint': None,
            'width': width,
            'height': height,
            'nucleoli': [],
            'mask_count': None,
            'pr_minval': None, 
            'pr_maxval': None, 
            'pr_sumval': None,
            'pr_meanval': None,
            'pr_variance': None,
            'pr_mt_correlation': None,
            'pr_er_correlation': None,
            'mt_minval': None, 
            'mt_maxval': None, 
            'mt_sumval': None,
            'mt_meanval': None,
            'mt_variance': None,
            'er_minval': None, 
            'er_maxval': None, 
            'er_sumval': None,
            'er_meanval': None,
            'er_variance': None,
            'nu_minval': None, 
            'nu_maxval': None, 
            'nu_sumval': None,
            'nu_meanval': None,
            'nu_variance': None,
            # values over the full cell
            'mask_count_fc': None,
            'pr_minval_fc': None,
            'pr_maxval_fc': None,
            'pr_sumval_fc': None,
            'pr_meanval_fc': None,
            'pr_variance_fc': None,
            'mt_minval_fc': None,
            'mt_maxval_fc': None,
            'mt_sumval_fc': None,
            'mt_meanval_fc': None,
            'mt_variance_fc': None,
            'pr_mt_correlation_fc': None,
            # values over the nucleus
            'nuc_mask_count': None,
            'nuc_pr_minval': None, 
            'nuc_pr_maxval': None, 
            'nuc_pr_sumval': None,
            'nuc_pr_meanval': None,
            'nuc_pr_variance': None,
            'nuc_mt_minval': None, 
            'nuc_mt_maxval': None, 
            'nuc_mt_sumval': None,
            'nuc_mt_meanval': None,
            'nuc_mt_variance': None,
            'nuc_nu_minval': None, 
            'nuc_nu_maxval': None, 
            'nuc_nu_sumval': None,
            'nuc_nu_meanval': None,
            'nuc_nu_variance': None,
            'nuc_nu_pr_correlation': None,
            # values over the nucleus inner rim
            'inrim_mask_count': None,
            'inrim_pr_minval': None,
            'inrim_pr_maxval': None,
            'inrim_pr_sumval': None,
            'inrim_pr_meanval': None,
            'inrim_pr_variance': None,
            'inrim_nu_minval': None,
            'inrim_nu_maxval': None,
            'inrim_nu_sumval': None,
            'inrim_nu_meanval': None,
            'inrim_nu_variance': None,
            'inrim_pr_nu_correlation': None,
            # values over the nucleus outer rim
            'outrim_mask_count': None,
            'outrim_pr_minval': None,
            'outrim_pr_maxval': None,
            'outrim_pr_sumval': None,
            'outrim_pr_meanval': None,
            'outrim_pr_variance': None,
            'outrim_nu_minval': None,
            'outrim_nu_maxval': None,
            'outrim_nu_sumval': None,
            'outrim_nu_meanval': None,
            'outrim_nu_variance': None,
            'outrim_pr_nu_correlation': None,
            'aggresome': False,
            'aggresome_size': None,
            #number of nuclear bodies
            'nucbody': 0,
            'nucbody_size': 0,  
            #number of nuclear Speckles
            'num_nucspeckles': 0,
            'nucspeckles_size': 0    
            }
    return cell

In [ ]:
def newNucleolusDict(image_id, image_label):

    nucleolus= {
            'image_id': image_id,
            'image_label': image_label,
            'mask_count': None,
            'cellnum': None,
            'arclength': None,
            'area': None,
            'bbox': None,
            'width': None,
            'height': None,
            'nucleoli': [],
            'pr_minval': None, 
            'pr_maxval': None, 
            'pr_sumval': None,
            'pr_meanval': None,
            'pr_variance': None,
            'pr_nu_correlation' : None,
            'nu_minval': None, 
            'nu_maxval': None, 
            'nu_sumval': None,
            'nu_meanval': None,
            'nu_variance': None
        }
    return nucleolus


In [ ]:
def newImageDict(image_id, trainlabel, width, height ):
    ann = {
            'image_id': image_id,
            'train_label': trainlabel,
            'width': width,
            'height': height,
            'cells': [],
            '141617-pred': None,
            'pr_minval': None, 
            'pr_maxval': None, 
            'pr_sumval': None,
            'pr_medianval': None,
            'pr_maxfreq': None,
            'pr_mostfreq': None,
            'mt_minval': None, 
            'mt_maxval': None, 
            'mt_sumval': None,
            'mt_medianval': None,
            'mt_maxfreq': None,
            'mt_mostfreq': None,
            'nu_minval': None, 
            'nu_maxval': None, 
            'nu_sumval': None,
            'nu_medianval': None,
            'nu_maxfreq': None,
            'nu_mostfreq': None,
            'er_minval': None, 
            'er_maxval': None, 
            'er_sumval': None,
            'er_medianval': None,
            'er_maxfreq': None,
            'er_mostfreq': None,
            'min_pr_er_correlation': None,
            'max_pr_er_correlation' : None,      
            'mean_pr_er_correlation': None,
            'min_pr_mt_correlation': None,
            'max_pr_mt_correlation' : None,      
            'mean_pr_mt_correlation': None,
            'min_pr_mt_correlation_fc': None,
            'max_pr_mt_correlation_fc': None,        
            'mean_pr_mt_correlation_fc': None
        }
    return ann

In [ ]:
def countTrainingScores(image_data_list,falsePos,falseNeg,truePos,trueNeg ):

    
    
    for image in image_data_list:
        image_label=image['train_label']
        expected_labels=image_label.split('|')

        
        for cell in image['cells']:

            results = cell['cell_results']
            for res in results:
                found=False
                for exp in expected_labels:
                    if int(exp) == int(res[0]):
                        found = True
                if found:
                    truePos[int(res[0])]+=1
                else:
                    falsePos[int(res[0])]+=1
            for exp in expected_labels:
                found=False
                for res in results:
                    if exp == res[0]:
                        found=True
                if not found:
                    falseNeg[int(exp)]+=1
                    
    print('True positives:')
    print(truePos)
    print('False positives:')
    print(falsePos)                    
    print('False negatives:')
    print(falseNeg)            

In [ ]:
def calculateScores(image_data):

    num_cells = len(image_data['cells'])
    #p = image_data['141617-pred']
    #print(f'Image level pred is {p}')

    for cell in image_data['cells']:
        cell_scored = False
        if cell['aggresome']:
            cell['cell_results'].append((15,max(cell['aggresome_size']/150,1)))
            cell_scored = True

            
        max_mt_corr = max(cell['pr_mt_correlation_fc'], cell['pr_mt_correlation_fc'] )
        if cell['pr_er_correlation'] > 0.4 or max_mt_corr > 0.4 :
            # Correlation with red or yellow layer found
            if cell['pr_er_correlation'] > max_mt_corr:
                cell['cell_results'].append((6,cell['pr_er_correlation']))
                cell_scored = True

            else:
                if cell['nuc_mt_meanval'] > 100:
                    cell['cell_results'].append((11,max_mt_corr))
                    cell_scored = True

                else:
                    cell['cell_results'].append((10,max_mt_corr))
                    cell_scored = True
                    
                    
    for cell in image_data['cells']:       
        max_mean = 0
        nucleolus_detected = False
        nucleolus_fcent_detected= False
        sum_nucleolus_mask_count=0
        sum_nucleolus_sums=0
        # Nucleolus
        for nucleolus in cell['nucleoli']:
            sum_nucleolus_mask_count += nucleolus['mask_count']
            sum_nucleolus_sums+=nucleolus['pr_sumval']
            if max_mean < nucleolus['pr_meanval']:
                max_mean = nucleolus['pr_meanval']
            if  nucleolus['pr_meanval'] > 150 \
                    and nucleolus['pr_meanval'] >  1.5 * cell['pr_meanval'] :
                if nucleolus['pr_variance'] < 1000:
                    nucleolus_detected = True
                else:
                    nucleolus_fcent_detected= True 
        
        nucleus_count_without_nucleoli = cell['nuc_mask_count']-sum_nucleolus_mask_count
        adjusted_nuc_mean = (cell['nuc_pr_sumval']-sum_nucleolus_sums)/nucleus_count_without_nucleoli

        npmeanval=cell['nuc_pr_meanval']
        im = cell['image_id']
        #if ( adjusted_nuc_mean < npmeanval):
            #print(f'Warning: {im} adjusted_nuc_mean {adjusted_nuc_mean} is less than cell nuc_pr_meanval {npmeanval}')
        
        if nucleolus_detected:
            cell['cell_results'].append((2,max_mean/255))
            cell_scored = True
        else:
            if nucleolus_fcent_detected and not cell['num_nucspeckles'] > 3:
                cell['cell_results'].append((3,max_mean/255))
                cell_scored = True
            else:
                if cell['num_nucspeckles'] > 0 :
                    cell['cell_results'].append((4,min(cell['nucspeckles_size']/60,1)))
                    cell_scored = True
                else:
                    if cell['nucbody'] > 0 and  cell['nucbody'] < 3:
                        cell['cell_results'].append((5,min(cell['nucbody_size']/18,1)))
                        cell_scored = True
                        # note (todo): possible nuc fib centre for nucbody > 3
                    else:
                        inr=cell['inrim_pr_meanval']
                        nucr=cell['nuc_pr_meanval']
                        if inr > nucr:
                            cell['cell_results'].append((1,(inr-nucr)/inr))
                            cell_scored = True
                        else:
                            if cell['inrim_pr_meanval'] > cell['outrim_pr_meanval']*1.1:
                                cell['cell_results'].append((0,adjusted_nuc_mean/255))
                                cell_scored = True

                                
        if image_data['141617-pred'] in ['14','16','17']:
            cell['cell_results'].append((image_data['141617-pred'],cell['pr_meanval']/255))
            cell_scored = True
            
        if not cell_scored:
            cell['cell_results'].append((18,1-(cell['pr_meanval']/255)))
        
        
                

In [ ]:
def maskStats( image_layer, mask_image, masknum, comp_image, statsdebug=False ): 
    # image_layer is a single colour layer (or slice of a layer) of the image # mask_image is an image (or slice) of the same size as image_layer with areas # set to different values indicating the number of the corresponding cell # as found in the ordering of the nucleus detection # The shape thus marked in the mask image can be, for instance: # - the area covered by the nucleus # - the area covered by the whole cell # - the area of the cell excluding the nucleus # - the area of a nucleolus # masknum is the value indicating the area of interest in the mask # comp_image is an image (or slice) of the same size to which image_layer is to be compared # Return the following calculated over all pixels with masknum at the same position in mask: # - minimum value of image_layer # - maximum value of image_layer # - sum of pixel values of image_layer # - mean value of image_layer # - variance of image_layer # - correlation coefficient between the layer and the comp_image

    # Work on a copies so as not to change the data external to the routine
    image_copy = np.copy(image_layer)
    comp_copy = np.copy(comp_image)
    
    if statsdebug:
        np.set_printoptions(threshold=sys.maxsize)
        print('image_copy')
        print(image_copy)
        print('comp_copy')
        print(comp_copy)
        print('mask_image')
        print(mask_image)
    
    # Find min (set the pixels not belonging to the area of interest to high values)
    image_copy[mask_image!=masknum]=255
    minval = image_copy.min()
    comp_copy[mask_image!=masknum]=255
    compmin = comp_copy.min()   

    # Find max (set the pixels not belonging to the area of interest to zero)
    image_copy[mask_image!=masknum]=0
    maxval = image_copy.max()
    comp_copy[mask_image!=masknum]=0
    compmax = comp_copy.max()

    # Count the number of relevant pixels
    numMaskPixels = np.sum(mask_image==masknum)

    # Calculate sum and mean pixel values for image and comp_image
    sumval = image_copy.sum()
    compsum = comp_copy.sum()
    if numMaskPixels == 0:
        meanval = 0
        compmean = 0
    else:
        meanval = sumval / numMaskPixels
        compmean = compsum / numMaskPixels  

    # Calculate variance, covariance for image and comp_image
    image_diff_mean = image_copy - meanval
    comp_diff_mean = comp_copy - compmean
    image_diff_mean[mask_image!=masknum]=0
    comp_diff_mean[mask_image!=masknum]=0
    
    image_sqdiffs = image_diff_mean**2
    comp_sqdiffs = comp_diff_mean**2

    if numMaskPixels == 0:
        im_variance = 0
        comp_variance = 0
    else:    
        im_variance = np.sum(image_sqdiffs)/numMaskPixels
        comp_variance = np.sum(comp_sqdiffs)/numMaskPixels

    diff_prod = image_diff_mean*comp_diff_mean

    if numMaskPixels == 0:
        covar = 0
    else:
        covar=np.sum(diff_prod)/numMaskPixels

    if im_variance > 0 and comp_variance > 0:
        corr = covar/math.sqrt(im_variance*comp_variance)
    else:
        corr = 0

    return minval, maxval, sumval, meanval, im_variance, compmin, \
        compmax, compsum, compmean, comp_variance, corr, numMaskPixels

In [ ]:
def analyseMicrotubules(image_data,contours,label,im,mt,pr, nuc_mask):

    min_correlation = 1
    max_correlation = -1
    sum_corr = 0  
    if label == '10':
        print(im)
        display_image_big(mt)
        display_image_big(pr)
    for j, (cell_contour, cell_data) in enumerate(zip(contours,image_data['cells'])):
        x,y,w,h = cell_data['bbox']
        full_image_mask=np.zeros(mt.shape, dtype='uint8')
        cv2.drawContours(full_image_mask, contours,j,255,cv2.FILLED,cv2.LINE_8)
        full_image_mask[nuc_mask==j+1]=0
        cropped_mt = mt[y:y+h, x:x+w]
        cropped_pr = pr[y:y+h, x:x+w]
        cropped_mask = full_image_mask[y:y+h, x:x+w]

        minval, maxval, sumval, meanval, im_variance, compmin, compmax,\
        compsum, compmean, comp_variance, corr, mask_count \
        = maskStats( cropped_pr, cropped_mask, 255, cropped_mt )

        cell_data['mask_count']=mask_count
        cell_data['pr_minval']=minval
        cell_data['pr_maxval']=maxval
        cell_data['pr_sumval']=sumval
        cell_data['pr_meanval']=meanval
        cell_data['pr_variance']=im_variance 
        cell_data['mt_minval']=compmin
        cell_data['mt_maxval']=compmax
        cell_data['mt_sumval']=compsum
        cell_data['mt_meanval']=compmean
        cell_data['mt_variance']=comp_variance
        cell_data['pr_mt_correlation']=corr

        sum_corr = sum_corr + corr
        if corr < min_correlation:
            min_correlation = corr
        if corr > max_correlation:
            max_correlation = corr
    mean_corr = sum_corr/len(contours)    
    image_data['min_pr_mt_correlation']= min_correlation
    image_data['max_pr_mt_correlation']= max_correlation        
    image_data['mean_pr_mt_correlation']= mean_corr

In [ ]:
def analyseMicrotubulesFullCell(image_data,contours,label,im,mt,pr, nuc_mask):

    min_correlation = 1
    max_correlation = -1
    sum_corr = 0  
    if label == '10':
        printinfo(im)
        display_image_big(mt)
        display_image_big(pr)
    for j, (cell_contour, cell_data) in enumerate(zip(contours,image_data['cells'])):
        x,y,w,h = cell_data['bbox']
        full_image_mask=np.zeros(mt.shape, dtype='uint8')
        cv2.drawContours(full_image_mask, contours,j,255,cv2.FILLED,cv2.LINE_8)

        cropped_mt = mt[y:y+h, x:x+w]
        cropped_pr = pr[y:y+h, x:x+w]
        cropped_mask = full_image_mask[y:y+h, x:x+w]

        minval, maxval, sumval, meanval, im_variance, compmin, compmax,\
        compsum, compmean, comp_variance, corr, mask_count \
        = maskStats( cropped_pr, cropped_mask, 255, cropped_mt )

        cell_data['mask_count_fc']=mask_count
        cell_data['pr_minval_fc']=minval
        cell_data['pr_maxval_fc']=maxval
        cell_data['pr_sumval_fc']=sumval
        cell_data['pr_meanval_fc']=meanval
        cell_data['pr_variance_fc']=im_variance 
        cell_data['mt_minval_fc']=compmin
        cell_data['mt_maxval_fc']=compmax
        cell_data['mt_sumval_fc']=compsum
        cell_data['mt_meanval_fc']=compmean
        cell_data['mt_variance_fc']=comp_variance
        cell_data['pr_mt_correlation_fc']=corr

        sum_corr = sum_corr + corr
        if corr < min_correlation:
            min_correlation = corr
        if corr > max_correlation:
            max_correlation = corr
    mean_corr = sum_corr/len(contours)    
    image_data['min_pr_mt_correlation_fc']= min_correlation
    image_data['max_pr_mt_correlation_fc']= max_correlation        
    image_data['mean_pr_mt_correlation_fc']= mean_corr

In [ ]:
def analyseER(image_data,contours,label,im,er,pr, nuc_mask):
    min_correlation = 1
    max_correlation = -1
    sum_corr = 0  

    for j, (cell_contour, cell_data) in enumerate(zip(contours,image_data['cells'])):
        x,y,w,h = cell_data['bbox']
        full_image_mask=np.zeros(er.shape, dtype='uint8')
        cv2.drawContours(full_image_mask, contours,j,255,cv2.FILLED,cv2.LINE_8)
        full_image_mask[nuc_mask==j+1]=0
        
        cropped_er = er[y:y+h, x:x+w]
        cropped_pr = pr[y:y+h, x:x+w]
        cropped_mask = full_image_mask[y:y+h, x:x+w]

        if False and debug:
            display_image_big(cropped_mask)
            display_image_big(cropped_pr)
            display_image_big(cropped_er)
        minval, maxval, sumval, meanval, im_variance, compmin, compmax,\
        compsum, compmean, comp_variance, corr, mask_count \
        = maskStats( cropped_pr, cropped_mask, 255, cropped_er )

        cell_data['mask_count']=mask_count
        cell_data['pr_minval']=minval
        cell_data['pr_maxval']=maxval
        cell_data['pr_sumval']=sumval
        cell_data['pr_meanval']=meanval
        cell_data['pr_variance']=im_variance 
        cell_data['er_minval']=compmin
        cell_data['er_maxval']=compmax
        cell_data['er_sumval']=compsum
        cell_data['er_meanval']=compmean
        cell_data['er_variance']=comp_variance
        cell_data['pr_er_correlation']=corr

        sum_corr = sum_corr + corr
        if corr < min_correlation:
            min_correlation = corr
        if corr > max_correlation:
            max_correlation = corr
    mean_corr = sum_corr/len(contours)    
    image_data['min_pr_er_correlation']= min_correlation
    image_data['max_pr_er_correlation']= max_correlation        
    image_data['mean_pr_er_correlation']= mean_corr

In [ ]:
def analyseNU(image_data,contours,label,im,nu,pr, nuc_mask):
  

    for j, (cell_contour, cell_data) in enumerate(zip(contours,image_data['cells'])):
        x,y,w,h = cell_data['bbox']
        full_image_mask=np.zeros(nu.shape, dtype='uint8')
        cv2.drawContours(full_image_mask, contours,j,255,cv2.FILLED,cv2.LINE_8)
        full_image_mask[nuc_mask==j+1]=0
        
        cropped_nu = nu[y:y+h, x:x+w]
        cropped_pr = pr[y:y+h, x:x+w]
        cropped_mask = full_image_mask[y:y+h, x:x+w]

        if False and debug:
            display_image_big(cropped_mask)
            display_image_big(cropped_pr)
            display_image_big(cropped_nu)
        minval, maxval, sumval, meanval, im_variance, compmin, compmax,\
        compsum, compmean, comp_variance, corr, mask_count \
        = maskStats( cropped_pr, cropped_mask, 255, cropped_nu )

        cell_data['nu_minval']=compmin
        cell_data['nu_maxval']=compmax
        cell_data['nu_sumval']=compsum
        cell_data['nu_meanval']=compmean
        cell_data['nu_variance']=comp_variance 


In [ ]:
def analyseMTNU(image_data,nuc_cont,label,im,nu,mt, nuc_mask):
  

    for j, (nuc_contour, cell_data) in enumerate(zip(nuc_cont,image_data['cells'])):
        x,y,w,h = cell_data['nuc_bbox']
        n_mask=np.zeros(nu.shape, dtype='uint8')
        cv2.drawContours(n_mask, nuc_cont,j,255,cv2.FILLED,cv2.LINE_8)
        
        cropped_nu = nu[y:y+h, x:x+w]
        cropped_mt = mt[y:y+h, x:x+w]
        cropped_mask = n_mask[y:y+h, x:x+w]

        if False and debug:
            display_image_big(cropped_mask)
            display_image_big(cropped_mt)
            display_image_big(cropped_nu)
        minval, maxval, sumval, meanval, im_variance, compmin, compmax,\
        compsum, compmean, comp_variance, corr, mask_count \
        = maskStats( cropped_mt, cropped_mask, 255, cropped_nu )
        cell_data['nuc_mt_minval']=compmin
        cell_data['nuc_mt_maxval']=compmax
        cell_data['nuc_mt_sumval']=compsum
        cell_data['nuc_mt_meanval']=compmean
        cell_data['nuc_mt_variance']=comp_variance  
  

In [ ]:
def generateNucRims(nuc_cont, nuc_midpoint, nu, rimwidth):
    # generate contours just rimwidth radius wider than the nucleus contours
    image_rim_contours=[]
    for j ,(cont, mid) in enumerate(zip(nuc_cont,nuc_midpoint)):

        #print(f'Contour {j}')
        #print(f'midpoint {mid}')

        newcont=[]
        
        imagesize=np.array(nu.shape)
        
        for k in range(len(cont)):
            #print(f'Contour {j} Segment {k}')
            vertex= nuc_cont[j][k][0]
            #print('vertex', vertex)
            rayvec = vertex - mid
            #print('rayvec', rayvec)
            #print(type(mid))
            scalefac=1/abs(rayvec).sum()
            rayvecnew=(rayvec*(abs(rayvec).sum()+rimwidth)*scalefac).astype(int)
            vertexnew=rayvecnew+mid
            clipvertex_low = np.fmax(vertexnew,[0,0]) 
            clipvertex = np.fmin(clipvertex_low,imagesize)
            newcont.append(clipvertex)
            #print('newcont', newcont)
        cnt=np.array(newcont)
        #print('nuc_cont',nuc_cont[j] )
        #print(type(cnt))
        #print('cnt', cnt)

        image_rim_contours.append(cnt)     
        #print(f'len(image_rim_contours), {len(image_rim_contours)}')
        #full_image_mask=np.zeros(nu.shape, dtype='uint8')        
        #cv2.drawContours(full_image_mask, image_rim_contours,-1,255,1)
        #cv2.drawContours(full_image_mask, nuc_cont,-1,200,1)
        #do_display_image_big(full_image_mask)
        

    return image_rim_contours

In [ ]:
def analyseNucleusInnerRims(image_data, nuc_cont, nuc_midpoint, nu, pr):
    INNER_NUCLEUS_RIMWIDTH = -10
    plus10cont=generateNucRims(nuc_cont, nuc_midpoint, nu, INNER_NUCLEUS_RIMWIDTH)

    for j, (nuc_contour, rim_contour, cell_data ) in \
        enumerate(zip(nuc_cont, plus10cont, image_data['cells'])):

        full_image_mask=np.zeros(nu.shape, dtype='uint8')
        cv2.drawContours(full_image_mask, nuc_cont,j,255,cv2.FILLED,cv2.LINE_8)
        cv2.drawContours(full_image_mask, plus10cont,j,0,cv2.FILLED,cv2.LINE_8)

        #if  debug:
            #do_display_image_big(full_image_mask) 
        
        x,y,w,h = cell_data['nuc_bbox']
        
        
        #print(f'x,y,w,h:{x},{y},{w},{h}')
        
        #cv2.rectangle(full_image_mask, (x,y), (x+w,y+h), 200,2)
        #display_image_big(full_image_mask) 
        cropped_mask = full_image_mask[y:y+h, x:x+w]
        cropped_nu = nu[y:y+h, x:x+w]
        cropped_pr = pr[y:y+h, x:x+w]
        

        #display_image_big(cropped_pr)
        #display_image_big(cropped_nu)
        
        minval, maxval, sumval, meanval, im_variance, compmin, compmax,\
        compsum, compmean, comp_variance, corr, mask_count \
        = maskStats( cropped_pr, cropped_mask, 255, cropped_nu )
        cell_data['inrim_mask_count']=mask_count
        cell_data['inrim_pr_minval']=minval
        cell_data['inrim_pr_maxval']=maxval
        cell_data['inrim_pr_sumval']=sumval
        cell_data['inrim_pr_meanval']=meanval
        cell_data['inrim_pr_variance']=im_variance 
        cell_data['inrim_nu_minval']=compmin
        cell_data['inrim_nu_maxval']=compmax
        cell_data['inrim_nu_sumval']=compsum
        cell_data['inrim_nu_meanval']=compmean
        cell_data['inrim_nu_variance']=comp_variance
        cell_data['inrim_pr_nu_correlation']=corr

In [ ]:
def overlaps(x1,y1,w1,h1,x2,y2,w2,h2):
    xseparate = x1+w1 < x2 or x2+w2 < x1
    yseparate = y1+h1 < y2 or y2+h2 < y1
    return not xseparate and not yseparate


In [ ]:
def lookForAggresomes (image_data, pr):
    MAXWIDTH=150
    MINWIDTH=30
    AREAMAX=22500
    AREAMIN=400
    
    prcopy = np.copy(pr)
    retval, prthresh= cv2.threshold(prcopy, 100,255,cv2.THRESH_BINARY)
    kernel = np.ones((5, 5), 'uint8')
    e=cv2.erode(prthresh,kernel,iterations=2)
    ed=cv2.dilate(e,kernel,iterations=5)
    ede=cv2.erode(ed,kernel,iterations=2)
    #do_display_image_big(ede)
    
    contours, hierarchy = cv2.findContours(ede, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    #print(f'number of contours: {len(contours)}')
    for i, cont in enumerate(contours):
        area = cv2.contourArea(cont)
        #print(f'area {area}')
        x,y,w,h = cv2.boundingRect(cont)
        #print('x,y,w,h',x,y,w,h)
        if area < AREAMAX and area > AREAMIN and hierarchy[0][i][3] == -1 \
            and w < MAXWIDTH and w > MINWIDTH \
            and h > int(w*0.8) and int(h*0.8) < w:
            # found potential age
            # Check whether it touches the bounding box of a cell
            #print('found contour of potential aggresome size, checking overlap with outrim ...')
            for j, cell_data in enumerate(image_data['cells']):
                cellx, celly, cellw, cellh = cell_data['bbox']
                #print('checking overlap with ',x,y,w,h,cellx, celly, cellw, cellh )
                if overlaps(x,y,w,h,cellx, celly, cellw, cellh):
                    #print('boxes overlap, checking masks')
                    outrim_mask=cell_data['outrim_mask']
                    binary_outrim_mask = outrim_mask.astype('bool')
                    #do_display_image_big(binary_outrim_mask)

                    full_image_mask=np.zeros(pr.shape, dtype='uint8')
                    cv2.drawContours(full_image_mask, contours,i,255,cv2.FILLED,cv2.LINE_8)
                    #do_display_image_big(full_image_mask)
                    contour_cell_slice=full_image_mask[celly:celly+cellh, cellx:cellx+cellw]
                    #do_display_image_big(contour_cell_slice)
                    contour_cell_binmask=contour_cell_slice.astype(bool)
                    #do_display_image_big(contour_cell_binmask)
                    show_mask = np.logical_or(binary_outrim_mask,contour_cell_binmask)
                    #do_display_image_big(show_mask)
                    result_mask = np.logical_and(binary_outrim_mask,contour_cell_binmask)
                    overlap_found=result_mask.max()
                    if overlap_found:
                        cell_data['aggresome']=True
                        cell_data['aggresome_size'] = w

In [ ]:
def lookForNuclearBodies (image_data, pr, nuc_mask):
    MAXWIDTH=14
    MINWIDTH=3
    AREAMAX=200
    AREAMIN=10
    
    num_cells=len(image_data['cells'])
    thresh = choose_threshold(pr, 2*num_cells*AREAMAX)
    prcopy = np.copy(pr)
    #do_display_image_big(pr)
    retval, prthresh= cv2.threshold(prcopy, thresh,255,cv2.THRESH_BINARY)
    kernel = np.ones((5, 5), 'uint8')
    e=cv2.erode(prthresh,kernel,iterations=1)
    #do_display_image_big(e)
    ed=cv2.dilate(e,kernel,iterations=2)
    #do_display_image_big(ed)
    ede=cv2.erode(ed,kernel,iterations=1)
    #do_display_image_big(ede)
    
    

    num_pixels=int(ede.sum()/255)

    # only perform on sparse arrays (dull images), not to confuse with nucleoli
    if num_pixels < 2*num_cells*AREAMAX:
        contours, hierarchy = cv2.findContours(ede, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        #print(f'number of contours: {len(contours)}')
        for i, cont in enumerate(contours):
            area = cv2.contourArea(cont)
            #print(f'area {area}')
            x,y,w,h = cv2.boundingRect(cont)
            #print('x,y,w,h',x,y,w,h)
            if area < AREAMAX and area > AREAMIN and hierarchy[0][i][3] == -1 \
                and w < MAXWIDTH and w > MINWIDTH \
                and h > int(w*0.6) and int(h*0.6) < w:
                # found potential nuclear body
                # Check whether it touches the bounding box of a cell
                #print('found contour of potential aggresome size, checking overlap with outrim ...')
                for j, cell_data in enumerate(image_data['cells']):
                    cellx, celly, cellw, cellh = cell_data['nuc_bbox']
                    #print('checking overlap with ',x,y,w,h,cellx, celly, cellw, cellh )
                    if overlaps(x,y,w,h,cellx, celly, cellw, cellh):
                        #print('boxes overlap, checking masks')
                        nuc_mask_slice=nuc_mask[celly:celly+cellh, cellx:cellx+cellw]
                        binary_nuc_mask = nuc_mask_slice.astype('bool')
                        #do_display_image_big(binary_outrim_mask)

                        full_image_mask=np.zeros(pr.shape, dtype='uint8')
                        cv2.drawContours(full_image_mask, contours,i,255,cv2.FILLED,cv2.LINE_8)
                        #do_display_image_big(full_image_mask)
                        contour_cell_slice=full_image_mask[celly:celly+cellh, cellx:cellx+cellw]
                        #do_display_image_big(contour_cell_slice)
                        contour_cell_binmask=contour_cell_slice.astype(bool)
                        #do_display_image_big(contour_cell_binmask)
                        show_mask = np.bitwise_xor(binary_nuc_mask,contour_cell_binmask)
                        #do_display_image_big(show_mask)
                        result_mask = np.logical_and(binary_nuc_mask,contour_cell_binmask)
                        overlap_found=result_mask.max()
                        if overlap_found:
                            cell_data['nucbody']+=1
                            cell_data['nucbody_size'] = w

In [ ]:
def lookForNuclearSpeckles (image_data, pr, nuc_mask):
    MAXWIDTH=50
    MINWIDTH=20
    AREAMAX=1500
    AREAMIN=100
    
    num_cells=len(image_data['cells'])
    thresh = choose_threshold(pr, 8*num_cells*AREAMAX)
    prcopy = np.copy(pr)
    #do_display_image_big(pr)
    retval, prthresh= cv2.threshold(prcopy, thresh,255,cv2.THRESH_BINARY)
    kernel = np.ones((5, 5), 'uint8')
    e=cv2.erode(prthresh,kernel,iterations=2)
    #do_display_image_big(e)
    ed=cv2.dilate(e,kernel,iterations=2)
    #do_display_image_big(ed)
    ede=ed
    #cv2.erode(ed,kernel,iterations=1)
    #do_display_image_big(ede)
    
    

    num_pixels=int(ede.sum()/255)

    # only perform on sparse arrays (dull images), not to confuse with nucleoli
    if num_pixels < 8*num_cells*AREAMAX:
        contours, hierarchy = cv2.findContours(ede, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        #print(f'number of contours: {len(contours)}')
        for i, cont in enumerate(contours):
            area = cv2.contourArea(cont)
            #print(f'area {area}')
            x,y,w,h = cv2.boundingRect(cont)
            #print('x,y,w,h',x,y,w,h)
            if area < AREAMAX and area > AREAMIN and hierarchy[0][i][3] == -1 \
                and w < MAXWIDTH and w > MINWIDTH \
                and h > int(w*0.3) and int(h*0.3) < w:
                # found potential nuclear body
                # Check whether it touches the bounding box of a cell
                #print('found contour of potential aggresome size, checking overlap with outrim ...')
                for j, cell_data in enumerate(image_data['cells']):
                    cellx, celly, cellw, cellh = cell_data['nuc_bbox']
                    #print('checking overlap with ',x,y,w,h,cellx, celly, cellw, cellh )
                    if overlaps(x,y,w,h,cellx, celly, cellw, cellh):
                        #print('boxes overlap, checking masks')
                        nuc_mask_slice=nuc_mask[celly:celly+cellh, cellx:cellx+cellw]
                        binary_nuc_mask = nuc_mask_slice.astype('bool')
                        #do_display_image_big(binary_outrim_mask)

                        full_image_mask=np.zeros(pr.shape, dtype='uint8')
                        cv2.drawContours(full_image_mask, contours,i,255,cv2.FILLED,cv2.LINE_8)
                        #do_display_image_big(full_image_mask)
                        contour_cell_slice=full_image_mask[celly:celly+cellh, cellx:cellx+cellw]
                        #do_display_image_big(contour_cell_slice)
                        contour_cell_binmask=contour_cell_slice.astype(bool)
                        #do_display_image_big(contour_cell_binmask)
                        show_mask = np.bitwise_xor(binary_nuc_mask,contour_cell_binmask)
                        #do_display_image_big(show_mask)
                        result_mask = np.logical_and(binary_nuc_mask,contour_cell_binmask)
                        overlap_found=result_mask.max()
                        if overlap_found:
                            cell_data['num_nucspeckles']+=1
                            cell_data['nucspeckles_size'] = w

In [ ]:
def analyseNucleusOuterRims(image_data, nuc_cont, contours, nuc_midpoint, nu, pr):
    OUTER_NUCLEUS_RIMWIDTH = 40
    plus10cont=generateNucRims(nuc_cont, nuc_midpoint, nu, OUTER_NUCLEUS_RIMWIDTH)

    for j, (nuc_contour, rim_contour, cell_data, cell_contour) in \
        enumerate(zip(nuc_cont, plus10cont, image_data['cells'], contours)):

        full_image_mask=np.zeros(nu.shape, dtype='uint8')
        cv2.drawContours(full_image_mask, plus10cont,j,255,cv2.FILLED,cv2.LINE_8)
        cv2.drawContours(full_image_mask, nuc_cont,j,0,cv2.FILLED,cv2.LINE_8)
        
        #clip off anything that is outside the cell boundary
        cell_boundary_mask=np.zeros(nu.shape, dtype='uint8')
        cv2.drawContours(cell_boundary_mask, contours,j,255,cv2.FILLED,cv2.LINE_8)
        full_image_mask[cell_boundary_mask==0]=0
        
        if False and debug and j < 3:
            do_display_image_big(full_image_mask) 
        
        x,y,w,h = cell_data['bbox']
        
        
        #print(f'x,y,w,h:{x},{y},{w},{h}')
        
        #cv2.rectangle(full_image_mask, (x,y), (x+w,y+h), 200,2)
        #display_image_big(full_image_mask) 
        cropped_mask = full_image_mask[y:y+h, x:x+w]
        cropped_nu = nu[y:y+h, x:x+w]
        cropped_pr = pr[y:y+h, x:x+w]
        

        #display_image_big(cropped_pr)
        #display_image_big(cropped_nu)
        
        minval, maxval, sumval, meanval, im_variance, compmin, compmax,\
        compsum, compmean, comp_variance, corr, mask_count \
        = maskStats( cropped_pr, cropped_mask, 255, cropped_nu )
        cell_data['outrim_mask_count']=mask_count
        cell_data['outrim_pr_minval']=minval
        cell_data['outrim_pr_maxval']=maxval
        cell_data['outrim_pr_sumval']=sumval
        cell_data['outrim_pr_meanval']=meanval
        cell_data['outrim_pr_variance']=im_variance 
        cell_data['outrim_nu_minval']=compmin
        cell_data['outrim_nu_maxval']=compmax
        cell_data['outrim_nu_sumval']=compsum
        cell_data['outrim_nu_meanval']=compmean
        cell_data['outrim_nu_variance']=comp_variance
        cell_data['outrim_pr_nu_correlation']=corr
        cell_data['outrim_mask']=np.copy(cropped_mask)

In [ ]:
def analyseNucleus( image_data, label, im, nu, er, mt, pr, nuc_cont, nuc_midpoint, nuc_mask, nuc_rect):   

    np.set_printoptions(threshold=sys.maxsize)

    prnuc = np.zeros(nu.shape, dtype='uint8')
    
    for j, (cont, cell_data) in enumerate(zip(nuc_cont,image_data['cells'])):
        x,y,w,h = nuc_rect[j]
        mtbc = np.copy(mt)       
        mtbc[nuc_mask!=j+1]=0
        erbc = np.copy(er)       
        erbc[nuc_mask!=j+1]=0
        nubc = np.copy(nu)       
        nubc[nuc_mask!=j+1]=0
        nubcCont = np.copy(nubc) 
        nubcContBlur = cv2.GaussianBlur(nubcCont, (5, 5), 0)
        prbc = np.copy(pr)       
        prbc[nuc_mask!=j+1]=0
        prbcCont = np.copy(prbc) 
        prbcContBlur = cv2.GaussianBlur(prbcCont, (5, 5), 0)
            
        cropped_mt = mtbc[y:y+h, x:x+w]
        cropped_er = erbc[y:y+h, x:x+w]
        cropped_nu = nubc[y:y+h, x:x+w]
        cropped_pr = prbc[y:y+h, x:x+w]
        cropped_mask = nuc_mask[y:y+h, x:x+w]

        minval, maxval, sumval, meanval, im_variance,\
        compmin, compmax, compsum, compmean, comp_variance, corr, mask_count \
        = maskStats( pr, nuc_mask, j+1, nu )

        cell_data['nuc_mask_count']=mask_count
        cell_data['nuc_pr_minval']=minval 
        cell_data['nuc_pr_maxval']=maxval 
        cell_data['nuc_pr_sumval']=sumval
        cell_data['nuc_pr_meanval']=meanval
        cell_data['nuc_pr_variance']=im_variance
        cell_data['nuc_nu_minval']=compmin
        cell_data['nuc_nu_maxval']=compmax
        cell_data['nuc_nu_sumval']=compsum
        cell_data['nuc_nu_meanval']=compmean
        cell_data['nuc_nu_variance']=comp_variance
        cell_data['nuc_nu_pr_correlation']=corr
      
 
                
        nuc_contour_area = cv2.contourArea(nuc_cont[j])
        cv2.drawContours(nubcContBlur, nuc_cont,j,255,1)
        cropped_nubcCont = nubcContBlur[y:y+h, x:x+w]                       
       
        mtMax = cropped_mt.max()
        erMax = cropped_er.max()
        nuMax = cropped_nu.max()
        prMax = cropped_pr.max()
        
        numMaskPixels = np.sum(nuc_mask==j+1)
            
        cropped_prsum = np.sum(cropped_pr)
        cropped_nusum = np.sum(cropped_nu)
        if numMaskPixels == 0:
            prMean = 0
            nuMean = 0
            nu_variance = 0
            pr_variance = 0            
            pearson_nupr = 0
        else:
            prMean = cropped_prsum / (numMaskPixels)
            nuMean = cropped_nusum / (numMaskPixels) 
            
            nuDiffMean = cropped_nu - nuMean
            nuDiffMean[cropped_mask!=j+1]=0
            prDiffMean = cropped_pr - prMean
            

            
            prDiffMean[cropped_mask!=j+1]=0
            

            nuprDiffProd=nuDiffMean*prDiffMean
            
            nuprcovariance=np.sum(nuprDiffProd) / (numMaskPixels)             
            
            nu_sqdiffs = nuDiffMean**2
            nu_sqdiffs[cropped_mask!=j+1]=0
            nu_variance = np.sum(nu_sqdiffs)/numMaskPixels

            pr_sqdiffs = prDiffMean**2
            pr_sqdiffs[cropped_mask!=j+1]=0
            pr_variance = np.sum(pr_sqdiffs)/numMaskPixels
            
            if nu_variance*pr_variance > 0.001:
                pearson_nupr = nuprcovariance / math.sqrt(nu_variance*pr_variance)
            else:
                pearson_nupr = 0

            #if j == 1:
                #print(f'matrix shape :  {cropped_pr.shape}')
                #print(f'nuMean :  {nuMean}')
                #print(f'numMaskPixels :  {numMaskPixels}')
                #print(f'nuprcovariance : , {nuprcovariance}')
                #print(f'nu_variance : , {nu_variance}')
                #print(f'pr_variance : , {pr_variance}')
                #print(f'pearson_nupr : {pearson_nupr}')

                #print(f'nuDiffMean[20][60] : {nuDiffMean[20][60]}')
                #print(f'prDiffMean[20][60] : {prDiffMean[20][60]}')
                #print(f'nuprDiffProd[20][60] : {nuprDiffProd[20][60]}')
                #print(f'nu_sqdiffs[20][60] : {nu_sqdiffs[20][60]}')
                #print(f'pr_sqdiffs[20][60] : {pr_sqdiffs[20][60]}')
                #print(nuprDiffProd)
                #print(nu_sqdiffs)
                #print(pr_sqdiffs)
        nucleoliPr=np.copy(cropped_pr)
        
        #if label == '14':
            #print(f'{label}, {im}, cell:{j}')
            #print('nucleus')
            #do_display_image_big(cropped_nubcCont)
            #print('protein in nucleus')
            #do_display_image_big(cropped_pr)

            
        cropped_nuc_thresh = cv2.adaptiveThreshold(cropped_nubcCont,255,cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY,201,0)
        display_image_big(cropped_nuc_thresh)          
        contours, hierarchy = cv2.findContours(cropped_nuc_thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        
        cropped_pr_thresh = cv2.adaptiveThreshold(cropped_pr,255,cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY,201,0)          
        pr_contours, pr_hierarchy = cv2.findContours(cropped_pr_thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

        num_nu_contours = len(contours)
        printinfo(f'num nu_contours: {num_nu_contours}' )

        sum_nuc_arclength = 0
        sum_nuc_area = 0
        min_nuc_area = 1000000
        max_nuc_area = 0
        for yy in contours:   
            nuc_arclength = cv2.arcLength(yy,True)
            nuc_area = cv2.contourArea(yy)
            if min_nuc_area > nuc_area :
                min_nuc_area = nuc_area
            if max_nuc_area < nuc_area :
                max_nuc_area = nuc_area          
            sum_nuc_arclength = sum_nuc_arclength + nuc_arclength
            sum_nuc_area = sum_nuc_area + nuc_area
        if num_nu_contours > 0:
            mean_nuc_arclength = sum_nuc_arclength / num_nu_contours
            mean_nuc_area = sum_nuc_area / num_nu_contours
        if train_or_test == 'train':
            with open('nucstats.txt', 'a') as outf:
                print(f'nucstats: num_nu_contours: {num_nu_contours}, sum_nuc_arclength: {sum_nuc_arclength}, sum_nuc_area: {sum_nuc_area} \
                mean_nuc_arclength: {mean_nuc_arclength}, mean_nuc_area: {mean_nuc_area} \
                min_nuc_area: {min_nuc_area}, max_nuc_area: {max_nuc_area} ', file=outf)
            printinfo(f'nucstats: num_nu_contours: {num_nu_contours}, sum_nuc_arclength: {sum_nuc_arclength}, sum_nuc_area: {sum_nuc_area} \
            mean_nuc_arclength: {mean_nuc_arclength}, mean_nuc_area: {mean_nuc_area} \
            min_nuc_area: {min_nuc_area}, max_nuc_area: {max_nuc_area} ')
                
        
        num_pr_contours = len(pr_contours)
        printinfo(f'num pr_contours: {num_pr_contours}' )

        sum_prc_arclength = 0
        sum_prc_area = 0
        min_prc_area = 1000000
        max_prc_area = 0
        for y in pr_contours:   
            prc_arclength = cv2.arcLength(y,True)
            prc_area = cv2.contourArea(y)
            if min_prc_area > prc_area :
                min_prc_area = prc_area
            if max_prc_area < prc_area :
                max_prc_area = prc_area          
            sum_prc_arclength = sum_prc_arclength + prc_arclength
            sum_prc_area = sum_prc_area + prc_area
        if num_pr_contours > 0:
            mean_prc_arclength = sum_prc_arclength / num_pr_contours
            mean_prc_area = sum_prc_area / num_pr_contours
        if train_or_test == 'train':
            with open('nucstats.txt', 'a') as outf:
                print(f'prcstats: num_pr_contours: {num_pr_contours}, sum_prc_arclength: {sum_prc_arclength}, sum_prc_area: {sum_prc_area} \
                mean_prc_arclength: {mean_prc_arclength}, mean_prc_area: {mean_prc_area} \
                min_prc_area: {min_prc_area}, max_prc_area: {max_prc_area} ', file=outf)
            printinfo(f'prcstats: num_pr_contours: {num_pr_contours}, sum_prc_arclength: {sum_prc_arclength}, sum_prc_area: {sum_prc_area} \
            mean_prc_arclength: {mean_prc_arclength}, mean_prc_area: {mean_prc_area} \
            min_prc_area: {min_prc_area}, max_prc_area: {max_prc_area} ')
        
        i=0
        num_contours_kept=0
        nuc_inner_cont=[]
        nuc_inner_midpoint=[]

        nucleolus_detected= False

        nucleolus_fcent_detected= False

        nuc_inner_mask = np.zeros(cropped_nu.shape, dtype='uint8')
        nucleoli_mask = np.zeros(cropped_nu.shape, dtype='uint8')
        
        max_nucleolus_pr_mean = 0
        # max_nucleolus_pr_mean tracks the maximum mean value of the protein within a recognised nucleolus in this cell
        # A correctly recognised nucleolus that has the nucleolus protein marking tends to have a high mean
        # therefore this is a first stab at a confidence level for the nucleoli detection
            
        for x in contours:
            nucleolus_found= False
            nucleolus_fcent_found = False
            
            arclength = cv2.arcLength(contours[i],True)
            area = cv2.contourArea(contours[i])

            #print(f'arclength:{arclength} hierarchy {hierarchy[0][i][3]}')
            if ( arclength > 20 and hierarchy[0][i][3] != -1 and area/nuc_contour_area < 0.25 ):
                num_contours_kept+=1
                nucleolus_data = newNucleolusDict(im,label)
                nucleolus_data['cellnum'] = cell_data['cellnum']
                nucleolus_data['arclength'] = arclength
                nucleolus_data['area'] = area
                nucleolus_data['nucleolus_num'] = num_contours_kept 
                cell_data['nucleoli'].append(nucleolus_data)
                
                nuc_inner_cont.append(x)


                cv2.drawContours(nuc_inner_mask, nuc_inner_cont,num_contours_kept-1,num_contours_kept,cv2.FILLED,cv2.LINE_8)
                cv2.drawContours(nucleoli_mask, nuc_inner_cont,num_contours_kept-1,255,cv2.FILLED,cv2.LINE_8)
                x,y,w,h = cv2.boundingRect(contours[i])
                nlbc = np.copy(cropped_nubcCont[y:y+h, x:x+w])
                nlbcpr = np.copy(cropped_pr[y:y+h, x:x+w])
                nlbc_mask = np.copy(nuc_inner_mask[y:y+h, x:x+w])
                nlbc[nlbc_mask!=num_contours_kept]=0
                nlbcpr[nlbc_mask!=num_contours_kept]=0          

                #if num_contours_kept == 21:
                #    lookclose = True
                #else:
                lookclose = False
                
                minval, maxval, sumval, meanval, im_variance,\
                compmin, compmax, compsum, compmean, comp_variance, corr, mask_count \
                = maskStats( nlbcpr, nlbc_mask, num_contours_kept, nlbc, statsdebug = lookclose )
                
                if False and debug and debug_show_nucleolus_masks:
                    print(area, meanval, i, lookclose)
                    displaycopy = np.copy(cropped_pr)
                    cv2.drawContours(displaycopy,nuc_inner_cont,num_contours_kept-1,255,1)
                    do_display_image_big(displaycopy)
                    
                nucleolus_data['mask_count']=mask_count
                nucleolus_data['pr_minval']=minval
                nucleolus_data['pr_maxval']=maxval
                nucleolus_data['pr_sumval']=sumval
                nucleolus_data['pr_meanval']=meanval
                nucleolus_data['pr_variance']=im_variance 
                nucleolus_data['nu_minval']=compmin
                nucleolus_data['nu_maxval']=compmax
                nucleolus_data['nu_sumval']=compsum
                nucleolus_data['nu_meanval']=compmean
                nucleolus_data['nu_variance']=comp_variance
                nucleolus_data['pr_nu_correlation']=corr                
                nucleolus_data['bbox'] = (x,y,w,h)                    

            i=i+1


In [ ]:
def showHistogram(bgrImage):
    bgr_planes = cv2.split(bgrImage)
    histSize = 256
    histRange = (0, 256) # the upper boundary is exclusive
    accumulate = False
    b_hist = cv2.calcHist(bgr_planes, [0], None, [histSize], histRange, accumulate=accumulate)
    g_hist = cv2.calcHist(bgr_planes, [1], None, [histSize], histRange, accumulate=accumulate)
    r_hist = cv2.calcHist(bgr_planes, [2], None, [histSize], histRange, accumulate=accumulate)
    hist_w = 512
    hist_h = 400
    bin_w = int(round( hist_w/histSize ))
    histImage = np.zeros((hist_h, hist_w, 3), dtype=np.uint8)
    cv2.normalize(b_hist, b_hist, alpha=0, beta=hist_h, norm_type=cv2.NORM_MINMAX)
    cv2.normalize(g_hist, g_hist, alpha=0, beta=hist_h, norm_type=cv2.NORM_MINMAX)
    cv2.normalize(r_hist, r_hist, alpha=0, beta=hist_h, norm_type=cv2.NORM_MINMAX)
    for i in range(1, histSize):
        cv2.line(histImage, ( bin_w*(i-1), hist_h - int(b_hist[i-1]) ),
            ( bin_w*(i), hist_h - int(b_hist[i]) ),
            ( 255, 0, 0), thickness=2)
        cv2.line(histImage, ( bin_w*(i-1), hist_h - int(g_hist[i-1]) ),
            ( bin_w*(i), hist_h - int(g_hist[i]) ),
            ( 0, 255, 0), thickness=2)
        cv2.line(histImage, ( bin_w*(i-1), hist_h - int(r_hist[i-1]) ),
            ( bin_w*(i), hist_h - int(r_hist[i]) ),
            ( 0, 0, 255), thickness=2)
    do_display_image_big(histImage)

In [ ]:
def printhist(im):
    hist_im = np.histogram(im, bins=[0,31,63,95,127,159,191,223,255])
    hist256=np.histogram(im, bins=256)
    maxbuck=0
    maxfreq=0
    for i in range(256):
        if hist256[0][i]>maxfreq:
            maxfreq=hist256[0][i]
            maxbuck=i
    print(hist_im)
    print(f'most frequent value is {maxbuck}')

In [ ]:
from skimage import feature
import numpy as np
class LocalBinaryPatterns:
    def __init__(self, numPoints, radius):
        # store the number of points and radius
        self.numPoints = numPoints
        self.radius = radius
    def describe(self, image, eps=1e-7):
        # compute the Local Binary Pattern representation
        # of the image, and then use the LBP representation
        # to build the histogram of patterns
        lbp = feature.local_binary_pattern(image, self.numPoints,
            self.radius, method="uniform")
        (hist, _) = np.histogram(lbp.ravel(),
            bins=np.arange(0, self.numPoints + 3),
            range=(0, self.numPoints + 2))
        # normalize the histogram
        hist = hist.astype("float")
        hist /= (hist.sum() + eps)
        # return the histogram of Local Binary Patterns
        return hist
    

In [ ]:
from sklearn.svm import LinearSVC

def addLbpData (desc, image_data, box_sample_list, pr, data, labels ):
    boxwidth=40
    label=image_data['train_label']
    max_boxes_per_image=3
    
    nn0=int(len(box_sample_list)/3)
    nn=nn0
    for b in box_sample_list:
        nn=nn+1
        if nn > nn0+max_boxes_per_image:
            break
        bllx=b[0]
        blly=b[1]
        image_slice = np.copy(pr[blly:blly+boxwidth, bllx:bllx+boxwidth])
        hist = desc.describe(image_slice)
        data.append(hist)
        print(f'adding with label {label}')
        do_display_image_big(image_slice)
        labels.append(label)
        

In [ ]:
def predictBoxlist(desc, model, image_data, box_sample_list, pr ):
    boxwidth=40
    label=image_data['train_label']
    
    preds=[]
    
    for b in box_sample_list:
        bllx=b[0]
        blly=b[1]
        image_slice = np.copy(pr[blly:blly+boxwidth, bllx:bllx+boxwidth])        
        hist = desc.describe(image_slice)
        prediction = model.predict(hist.reshape(1, -1))
        #print(f'prediction {prediction[0]} label {label}')
        found=False

        for pp, pred in enumerate( preds ):
            if prediction[0]==pred[0]:
                preds[pp]=(prediction[0],pred[1]+1)
                found=True
        if not found:
            preds.append((prediction[0],1))
    #printinfo('List result:')
    #printinfo(preds)
    nummaxpred=0
    maxpred='18'
    for pred in preds:
        if pred[1]>nummaxpred:
            maxpred = pred[0]
            nummaxpred=pred[1]
    image_data['141617-pred']=maxpred
    #printinfo(f'train label is {label}, Result is {maxpred}')

In [ ]:
def compassSampleBoxes(image_data, nuc_cont, contours, nuc_mask, nuc_midpoint, image):
    contour_box_ll_list=[]
    rimwidth=70
    boxwidth=40
    drawcopy = np.copy(image)

    cv2.drawContours(drawcopy, nuc_cont,-1,255,1)
    cv2.drawContours(drawcopy, contours,-1,255,1)
    
    for j ,(cont, mid, cellcont) in enumerate(zip(nuc_cont,nuc_midpoint,contours)):

        #print(f'Contour {j}')
        #print(f'midpoint {mid}')
        test_mask=np.zeros((image.shape[0],image.shape[1]), np.uint8)
        cv2.drawContours(test_mask,contours,j,255,cv2.FILLED,cv2.LINE_8)
        cv2.drawContours(test_mask,nuc_cont,j,0,cv2.FILLED,cv2.LINE_8)

        imagesize=np.array(image.shape)
        
        for k in range(len(cont)):
            #print(f'Contour {j} Segment {k}')
            vertex= nuc_cont[j][k][0]
            #print('vertex', vertex)
            rayvec = vertex - mid
            #print('rayvec', rayvec)
            #print(type(mid))
            scalefac=1/abs(rayvec).sum()
            rayvecnew=(rayvec*(abs(rayvec).sum()+rimwidth)*scalefac).astype(int)
            vertexnew=rayvecnew+mid
            clipvertex_low = np.fmax(vertexnew,[0,0]) 
            clipvertex = np.fmin(clipvertex_low,imagesize)  
            
            min_point_x=min(vertex[0],clipvertex[0])
            min_point_y=min(vertex[1],clipvertex[1])
            
            #if  j == 4 :           
                #if (test_box_slice.sum()/255 > boxwidth*boxwidth*0.8 ) :           
                #    cv2.rectangle(test_mask,(min_point_x, min_point_y),(min_point_x+boxwidth, min_point_y+boxwidth),125, 2 )
                #else: 
                #    cv2.rectangle(test_mask,(min_point_x, min_point_y),(min_point_x+boxwidth, min_point_y+boxwidth),20, 2 )
            
            
            test_box_slice=np.copy(test_mask[min_point_y:min_point_y+boxwidth, min_point_x:min_point_x+boxwidth]) 
            tbss=test_box_slice.sum()/255

            if ( tbss > boxwidth*boxwidth*0.9 and \
                min_point_x+boxwidth < imagesize[0] and min_point_y+boxwidth < imagesize[1]):
                    image_slice = np.copy(image[min_point_y:min_point_y+boxwidth, min_point_x:min_point_x+boxwidth])

                    info_content=image_slice.sum()/255
                    if (info_content > 20):# remove almost-empty boxes
                        contour_box_ll_list.append((min_point_x, min_point_y))
                        #cv2.rectangle(drawcopy,(min_point_x, min_point_y),(min_point_x+boxwidth, min_point_y+boxwidth), 255, 2 )
                        #cv2.circle(drawcopy, (min_point_x, min_point_y), 5, 255, 2)

    #do_display_image_big(drawcopy)

    return contour_box_ll_list

In [ ]:
## percentpercentwritefile faster_hpa_cell_segment.py

import numpy as np
import pandas as pd
import os
from tqdm import tqdm

from hpacellseg.cellsegmentator import *


class CellSegmentator(object):
    """Uses pretrained DPN-Unet models to segment cells from images."""

    def __init__(
        self,
        nuclei_model="../input/hpacellsegmentatormodelweights/dpn_unet_nuclei_v1.pth",
        cell_model="../input/hpacellsegmentatormodelweights/dpn_unet_cell_3ch_v1.pth",
        scale_factor=1.0,
        device="cuda",
        padding=False,
        multi_channel_model=True,
    ):
        """Class for segmenting nuclei and whole cells from confocal microscopy images.
        It takes lists of images and returns the raw output from the
        specified segmentation model. Models can be automatically
        downloaded if they are not already available on the system.
        When working with images from the Huan Protein Cell atlas, the
        outputs from this class' methods are well combined with the
        label functions in the utils module.
        Note that for cell segmentation, there are two possible models
        available. One that works with 2 channeled images and one that
        takes 3 channels.
        Keyword arguments:
        nuclei_model -- A loaded torch nuclei segmentation model or the
                        path to a file which contains such a model.
                        If the argument is a path that points to a non-existant file,
                        a pretrained nuclei_model is going to get downloaded to the
                        specified path (default: './nuclei_model.pth').
        cell_model -- A loaded torch cell segmentation model or the
                      path to a file which contains such a model.
                      The cell_model argument can be None if only nuclei
                      are to be segmented (default: './cell_model.pth').
        scale_factor -- How much to scale images before they are fed to
                        segmentation models. Segmentations will be scaled back
                        up by 1/scale_factor to match the original image
                        (default: 0.25).
        device -- The device on which to run the models.
                  This should either be 'cpu' or 'cuda' or pointed cuda
                  device like 'cuda:0' (default: 'cuda').
        padding -- Whether to add padding to the images before feeding the
                   images to the network. (default: False).
        multi_channel_model -- Control whether to use the 3-channel cell model or not.
                               If True, use the 3-channel model, otherwise use the
                               2-channel version (default: True).
        """
        if device != "cuda" and device != "cpu" and "cuda" not in device:
            raise ValueError(f"{device} is not a valid device (cuda/cpu)")
        if device != "cpu":
            try:
                assert torch.cuda.is_available()
            except AssertionError:
                print("No GPU found, using CPU.", file=sys.stderr)
                device = "cpu"
        self.device = device

        if isinstance(nuclei_model, str):
            if not os.path.exists(nuclei_model):
                print(
                    f"Could not find {nuclei_model}. Downloading it now",
                    file=sys.stderr,
                )
                download_with_url(NUCLEI_MODEL_URL, nuclei_model)
            nuclei_model = torch.load(
                nuclei_model, map_location=torch.device(self.device)
            )
        if isinstance(nuclei_model, torch.nn.DataParallel) and device == "cpu":
            nuclei_model = nuclei_model.module

        self.nuclei_model = nuclei_model.to(self.device).eval()

        self.multi_channel_model = multi_channel_model
        if isinstance(cell_model, str):
            if not os.path.exists(cell_model):
                print(
                    f"Could not find {cell_model}. Downloading it now", file=sys.stderr
                )
                if self.multi_channel_model:
                    download_with_url(MULTI_CHANNEL_CELL_MODEL_URL, cell_model)
                else:
                    download_with_url(TWO_CHANNEL_CELL_MODEL_URL, cell_model)
            cell_model = torch.load(cell_model, map_location=torch.device(self.device))
        self.cell_model = cell_model.to(self.device).eval()
        self.scale_factor = scale_factor
        self.padding = padding

    def _image_conversion(self, images):
        """Convert/Format images to RGB image arrays list for cell predictions.
        Intended for internal use only.
        Keyword arguments:
        images -- list of lists of image paths/arrays. It should following the
                 pattern if with er channel input,
                 [
                     [microtubule_path0/image_array0, microtubule_path1/image_array1, ...],
                     [er_path0/image_array0, er_path1/image_array1, ...],
                     [nuclei_path0/image_array0, nuclei_path1/image_array1, ...]
                 ]
                 or if without er input,
                 [
                     [microtubule_path0/image_array0, microtubule_path1/image_array1, ...],
                     None,
                     [nuclei_path0/image_array0, nuclei_path1/image_array1, ...]
                 ]
        """
        microtubule_imgs, er_imgs, nuclei_imgs = images
        if self.multi_channel_model:
            if not isinstance(er_imgs, list):
                raise ValueError("Please speicify the image path(s) for er channels!")
        else:
            if not er_imgs is None:
                raise ValueError(
                    "second channel should be None for two channel model predition!"
                )

        if not isinstance(microtubule_imgs, list):
            raise ValueError("The microtubule images should be a list")
        if not isinstance(nuclei_imgs, list):
            raise ValueError("The microtubule images should be a list")

        if er_imgs:
            if not len(microtubule_imgs) == len(er_imgs) == len(nuclei_imgs):
                raise ValueError("The lists of images needs to be the same length")
        else:
            if not len(microtubule_imgs) == len(nuclei_imgs):
                raise ValueError("The lists of images needs to be the same length")

        if not all(isinstance(item, np.ndarray) for item in microtubule_imgs):
            microtubule_imgs = [
                os.path.expanduser(item) for _, item in enumerate(microtubule_imgs)
            ]
            nuclei_imgs = [
                os.path.expanduser(item) for _, item in enumerate(nuclei_imgs)
            ]

            microtubule_imgs = list(
                map(lambda item: imageio.imread(item), microtubule_imgs)
            )
            nuclei_imgs = list(map(lambda item: imageio.imread(item), nuclei_imgs))
            if er_imgs:
                er_imgs = [os.path.expanduser(item) for _, item in enumerate(er_imgs)]
                er_imgs = list(map(lambda item: imageio.imread(item), er_imgs))

        if not er_imgs:
            er_imgs = [
                np.zeros(item.shape, dtype=item.dtype)
                for _, item in enumerate(microtubule_imgs)
            ]
        cell_imgs = list(
            map(
                lambda item: np.dstack((item[0], item[1], item[2])),
                list(zip(microtubule_imgs, er_imgs, nuclei_imgs)),
            )
        )

        return cell_imgs

    def pred_nuclei(self, images):
        """Predict the nuclei segmentation.
        Keyword arguments:
        images -- A list of image arrays or a list of paths to images.
                  If as a list of image arrays, the images could be 2d images
                  of nuclei data array only, or must have the nuclei data in
                  the blue channel; If as a list of file paths, the images
                  could be RGB image files or gray scale nuclei image file
                  paths.
        Returns:
        predictions -- A list of predictions of nuclei segmentation for each nuclei image.
        """

        def _preprocess(image):
            if isinstance(image, str):
                image = imageio.imread(image)
            self.target_shape = image.shape
            if len(image.shape) == 2:
                image = np.dstack((image, image, image))
            image = transform.rescale(image, self.scale_factor, multichannel=True)
            nuc_image = np.dstack((image[..., 2], image[..., 2], image[..., 2]))
            if self.padding:
                rows, cols = nuc_image.shape[:2]
                self.scaled_shape = rows, cols
                nuc_image = cv2.copyMakeBorder(
                    nuc_image,
                    32,
                    (32 - rows % 32),
                    32,
                    (32 - cols % 32),
                    cv2.BORDER_REFLECT,
                )
            nuc_image = nuc_image.transpose([2, 0, 1])
            return nuc_image

        def _segment_helper(imgs):
            with torch.no_grad():
                mean = torch.as_tensor(NORMALIZE["mean"], device=self.device)
                std = torch.as_tensor(NORMALIZE["std"], device=self.device)
                imgs = torch.tensor(imgs).float()
                imgs = imgs.to(self.device)
                imgs = imgs.sub_(mean[:, None, None]).div_(std[:, None, None])

                imgs = self.nuclei_model(imgs)
                imgs = F.softmax(imgs, dim=1)
                return imgs

        preprocessed_imgs = list(map(_preprocess, images))
        bs = 24
        predictions = []
        for i in range(0, len(preprocessed_imgs), bs):
            start = i
            end = min(len(preprocessed_imgs), i+bs)
            x = preprocessed_imgs[start:end]
            pred = _segment_helper(x).cpu().numpy()
            predictions.append(pred)
        predictions = list(np.concatenate(predictions, axis=0))
        predictions = map(util.img_as_ubyte, predictions)
        predictions = list(map(self._restore_scaling_padding, predictions))
        return predictions

    def _restore_scaling_padding(self, n_prediction):
        """Restore an image from scaling and padding.
        This method is intended for internal use.
        It takes the output from the nuclei model as input.
        """
        n_prediction = n_prediction.transpose([1, 2, 0])
        if self.padding:
            n_prediction = n_prediction[
                32 : 32 + self.scaled_shape[0], 32 : 32 + self.scaled_shape[1], ...
            ]
        if not self.scale_factor == 1:
            n_prediction[..., 0] = 0
            n_prediction = cv2.resize(
                n_prediction,
                (self.target_shape[0], self.target_shape[1]),
                interpolation=cv2.INTER_AREA,
            )
        return n_prediction

    def pred_cells(self, images, precombined=False):
        """Predict the cell segmentation for a list of images.
        Keyword arguments:
        images -- list of lists of image paths/arrays. It should following the
                  pattern if with er channel input,
                  [
                      [microtubule_path0/image_array0, microtubule_path1/image_array1, ...],
                      [er_path0/image_array0, er_path1/image_array1, ...],
                      [nuclei_path0/image_array0, nuclei_path1/image_array1, ...]
                  ]
                  or if without er input,
                  [
                      [microtubule_path0/image_array0, microtubule_path1/image_array1, ...],
                      None,
                      [nuclei_path0/image_array0, nuclei_path1/image_array1, ...]
                  ]
                  The ER channel is required when multichannel is True
                  and required to be None when multichannel is False.
                  The images needs to be of the same size.
        precombined -- If precombined is True, the list of images is instead supposed to be
                       a list of RGB numpy arrays (default: False).
        Returns:
        predictions -- a list of predictions of cell segmentations.
        """

        def _preprocess(image):
            self.target_shape = image.shape
            if not len(image.shape) == 3:
                raise ValueError("image should has 3 channels")
            cell_image = transform.rescale(image, self.scale_factor, multichannel=True)
            if self.padding:
                rows, cols = cell_image.shape[:2]
                self.scaled_shape = rows, cols
                cell_image = cv2.copyMakeBorder(
                    cell_image,
                    32,
                    (32 - rows % 32),
                    32,
                    (32 - cols % 32),
                    cv2.BORDER_REFLECT,
                )
            cell_image = cell_image.transpose([2, 0, 1])
            return cell_image

        def _segment_helper(imgs):
            with torch.no_grad():
                mean = torch.as_tensor(NORMALIZE["mean"], device=self.device)
                std = torch.as_tensor(NORMALIZE["std"], device=self.device)
                imgs = torch.tensor(imgs).float()
                imgs = imgs.to(self.device)
                imgs = imgs.sub_(mean[:, None, None]).div_(std[:, None, None])

                imgs = self.cell_model(imgs)
                imgs = F.softmax(imgs, dim=1)
                return imgs

        if not precombined:
            images = self._image_conversion(images)
        preprocessed_imgs = list(map(_preprocess, images))
        bs = 24
        predictions = []
        for i in range(0, len(preprocessed_imgs), bs):
            start = i
            end = min(len(preprocessed_imgs), i+bs)
            x = preprocessed_imgs[start:end]
            pred = _segment_helper(x).cpu().numpy()
            predictions.append(pred)
        predictions = list(np.concatenate(predictions, axis=0))
        predictions = map(self._restore_scaling_padding, predictions)
        predictions = list(map(util.img_as_ubyte, predictions))

        return predictions
    
    


        
    


In [ ]:
import os.path
import urllib
import zipfile

import numpy as np
import scipy.ndimage as ndi
from skimage import filters, measure, segmentation
from skimage.morphology import (binary_erosion, closing, disk,
                                remove_small_holes, remove_small_objects)

HIGH_THRESHOLD = 0.4
LOW_THRESHOLD = HIGH_THRESHOLD - 0.25


def download_with_url(url_string, file_path, unzip=False):
    """Download file with a link."""
    with urllib.request.urlopen(url_string) as response, open(
        file_path, "wb"
    ) as out_file:
        data = response.read()  # a `bytes` object
        out_file.write(data)

    if unzip:
        with zipfile.ZipFile(file_path, "r") as zip_ref:
            zip_ref.extractall(os.path.dirname(file_path))


def __fill_holes(image):
    """Fill_holes for labelled image, with a unique number."""
    boundaries = segmentation.find_boundaries(image)
    image = np.multiply(image, np.invert(boundaries))
    image = ndi.binary_fill_holes(image > 0)
    image = ndi.label(image)[0]
    return image





def label_cell(nuclei_pred, cell_pred):
    """Label the cells and the nuclei.
    Keyword arguments:
    nuclei_pred -- a 3D numpy array of a prediction from a nuclei image.
    cell_pred -- a 3D numpy array of a prediction from a cell image.
    Returns:
    A tuple containing:
    nuclei-label -- A nuclei mask data array.
    cell-label  -- A cell mask data array.
    0's in the data arrays indicate background while a continous
    strech of a specific number indicates the area for a specific
    cell.
    The same value in cell mask and nuclei mask refers to the identical cell.
    NOTE: The nuclei labeling from this function will be sligthly
    different from the values in :func:`label_nuclei` as this version
    will use information from the cell-predictions to make better
    estimates.
    """
    def __wsh(
        mask_img,
        threshold,
        border_img,
        seeds,
        threshold_adjustment=0.35,
        small_object_size_cutoff=10,
    ):
        img_copy = np.copy(mask_img)
        m = seeds * border_img  # * dt
        img_copy[m <= threshold + threshold_adjustment] = 0
        img_copy[m > threshold + threshold_adjustment] = 1
        img_copy = img_copy.astype(np.bool)
        img_copy = remove_small_objects(img_copy, small_object_size_cutoff).astype(
            np.uint8
        )

        mask_img[mask_img <= threshold] = 0
        mask_img[mask_img > threshold] = 1
        mask_img = mask_img.astype(np.bool)
        mask_img = remove_small_holes(mask_img, 63)
        mask_img = remove_small_objects(mask_img, 1).astype(np.uint8)
        markers = ndi.label(img_copy, output=np.uint32)[0]
        labeled_array = segmentation.watershed(
            mask_img, markers, mask=mask_img, watershed_line=True
        )
        return labeled_array

    nuclei_label = __wsh(
        nuclei_pred[..., 2] / 255.0,
        0.4,
        1 - (nuclei_pred[..., 1] + cell_pred[..., 1]) / 255.0 > 0.05,
        nuclei_pred[..., 2] / 255,
        threshold_adjustment=-0.25,
        small_object_size_cutoff=32,
    )

    # for hpa_image, to remove the small pseduo nuclei
    nuclei_label = remove_small_objects(nuclei_label, 157)
    nuclei_label = measure.label(nuclei_label)
    # this is to remove the cell borders' signal from cell mask.
    # could use np.logical_and with some revision, to replace this func.
    # Tuned for segmentation hpa images
    threshold_value = max(0.22, filters.threshold_otsu(cell_pred[..., 2] / 255) * 0.5)
    # exclude the green area first
    cell_region = np.multiply(
        cell_pred[..., 2] / 255 > threshold_value,
        np.invert(np.asarray(cell_pred[..., 1] / 255 > 0.05, dtype=np.int8)),
    )
    sk = np.asarray(cell_region, dtype=np.int8)
    distance = np.clip(cell_pred[..., 2], 255 * threshold_value, cell_pred[..., 2])
    cell_label = segmentation.watershed(-distance, nuclei_label, mask=sk)
    cell_label = remove_small_objects(cell_label, 344).astype(np.uint8)
    selem = disk(2)
    cell_label = closing(cell_label, selem)
    cell_label = __fill_holes(cell_label)
    # this part is to use green channel, and extend cell label to green channel
    # benefit is to exclude cells clear on border but without nucleus
    sk = np.asarray(
        np.add(
            np.asarray(cell_label > 0, dtype=np.int8),
            np.asarray(cell_pred[..., 1] / 255 > 0.05, dtype=np.int8),
        )
        > 0,
        dtype=np.int8,
    )
    cell_label = segmentation.watershed(-distance, cell_label, mask=sk)
    cell_label = __fill_holes(cell_label)
    cell_label = np.asarray(cell_label > 0, dtype=np.uint8)
    cell_label = measure.label(cell_label)
    cell_label = remove_small_objects(cell_label, 344)
    cell_label = measure.label(cell_label)
    cell_label = np.asarray(cell_label, dtype=np.uint16)
    nuclei_label = np.multiply(cell_label > 0, nuclei_label) > 0
    nuclei_label = measure.label(nuclei_label)
    nuclei_label = remove_small_objects(nuclei_label, 157)
    nuclei_label = np.multiply(cell_label, nuclei_label > 0)

    return nuclei_label, cell_label


In [ ]:
def newImageDict(image_id, trainlabel, width, height ):
    ann = {
            'image_id': image_id,
            'train_label': trainlabel,
            'width': width,
            'height': height,
            'cells': [],
            'pr_minval': None, 
            'pr_maxval': None, 
            'pr_sumval': None,
            'pr_medianval': None,
            'pr_maxfreq': None,
            'pr_mostfreq': None,
            'mt_minval': None, 
            'mt_maxval': None, 
            'mt_sumval': None,
            'mt_medianval': None,
            'mt_maxfreq': None,
            'mt_mostfreq': None,
            'nu_minval': None, 
            'nu_maxval': None, 
            'nu_sumval': None,
            'nu_medianval': None,
            'nu_maxfreq': None,
            'nu_mostfreq': None,
            'er_minval': None, 
            'er_maxval': None, 
            'er_sumval': None,
            'er_medianval': None,
            'er_maxfreq': None,
            'er_mostfreq': None,
            'min_pr_er_correlation': None,
            'max_pr_er_correlation' : None,      
            'mean_pr_er_correlation': None,
            'min_pr_mt_correlation': None,
            'max_pr_mt_correlation' : None,      
            'mean_pr_mt_correlation': None,
            'min_pr_mt_correlation_fc': None,
            'max_pr_mt_correlation_fc': None,        
            'mean_pr_mt_correlation_fc': None
        }
    return ann

In [ ]:
class CellSegmentatorOpenCV(object):
    """Uses openCV functionality to identify nucleus and cell masks"""

    def __init__(
        self,
        nuclei_model="../input/hpacellsegmentatormodelweights/dpn_unet_nuclei_v1.pth",
        cell_model="../input/hpacellsegmentatormodelweights/dpn_unet_cell_3ch_v1.pth",
        scale_factor=1.0,
        device="cuda",
        padding=False,
        multi_channel_model=True,
    ):
        self.scale_factor = scale_factor
        self.padding = padding
   
    def _image_conversion(self, images):
        """Convert/Format images to RGB image arrays list for cell predictions.
        Intended for internal use only.
        Keyword arguments:
        images -- list of lists of image paths/arrays. It should following the
                 pattern if with er channel input,
                 [
                     [microtubule_path0/image_array0, microtubule_path1/image_array1, ...],
                     [er_path0/image_array0, er_path1/image_array1, ...],
                     [nuclei_path0/image_array0, nuclei_path1/image_array1, ...]
                 ]
                 or if without er input,
                 [
                     [microtubule_path0/image_array0, microtubule_path1/image_array1, ...],
                     None,
                     [nuclei_path0/image_array0, nuclei_path1/image_array1, ...]
                 ]
        """
        microtubule_imgs, er_imgs, nuclei_imgs = images
        if self.multi_channel_model:
            if not isinstance(er_imgs, list):
                raise ValueError("Please speicify the image path(s) for er channels!")
        else:
            if not er_imgs is None:
                raise ValueError(
                    "second channel should be None for two channel model predition!"
                )

        if not isinstance(microtubule_imgs, list):
            raise ValueError("The microtubule images should be a list")
        if not isinstance(nuclei_imgs, list):
            raise ValueError("The microtubule images should be a list")

        if er_imgs:
            if not len(microtubule_imgs) == len(er_imgs) == len(nuclei_imgs):
                raise ValueError("The lists of images needs to be the same length")
        else:
            if not len(microtubule_imgs) == len(nuclei_imgs):
                raise ValueError("The lists of images needs to be the same length")

        if not all(isinstance(item, np.ndarray) for item in microtubule_imgs):
            microtubule_imgs = [
                os.path.expanduser(item) for _, item in enumerate(microtubule_imgs)
            ]
            nuclei_imgs = [
                os.path.expanduser(item) for _, item in enumerate(nuclei_imgs)
            ]

            microtubule_imgs = list(
                map(lambda item: imageio.imread(item), microtubule_imgs)
            )
            nuclei_imgs = list(map(lambda item: imageio.imread(item), nuclei_imgs))
            if er_imgs:
                er_imgs = [os.path.expanduser(item) for _, item in enumerate(er_imgs)]
                er_imgs = list(map(lambda item: imageio.imread(item), er_imgs))

        if not er_imgs:
            er_imgs = [
                np.zeros(item.shape, dtype=item.dtype)
                for _, item in enumerate(microtubule_imgs)
            ]
        cell_imgs = list(
            map(
                lambda item: np.dstack((item[0], item[1], item[2])),
                list(zip(microtubule_imgs, er_imgs, nuclei_imgs)),
            )
        )

        return cell_imgs
    


    
    
    def pred_nuclei(self, images):
        """Predict the nuclei segmentation.
        Keyword arguments:
        images -- A list of image arrays or a list of paths to images.
                  If as a list of image arrays, the images could be 2d images
                  of nuclei data array only, or must have the nuclei data in
                  the blue channel; If as a list of file paths, the images
                  could be RGB image files or gray scale nuclei image file
                  paths.
        Returns:
        predictions -- A list of predictions of nuclei segmentation for each nuclei image.
        """
        
        def _preprocess(image):
  
            if isinstance(image, str):
                image = imageio.imread(image)

            if not self.scale_factor == 1:
                nuc_image = transform.rescale(image, self.scale_factor, multichannel=True)
            else:
                nuc_image = image

            if self.padding:
                rows, cols = nuc_image.shape
                self.scaled_shape = rows, cols
                nuc_image = cv2.copyMakeBorder(
                    nuc_image,
                    32,
                    (32 - rows % 32),
                    32,
                    (32 - cols % 32),
                    cv2.BORDER_REFLECT,
                )
            return nuc_image

        preprocessed_imgs = list(map(_preprocess, images))

        predictions = []
        for i in range(len(preprocessed_imgs)):
            nuc_cont, nuc_midpoint, nuc_mask, nuc_rect = find_nuc_contours (preprocessed_imgs[i])
            predictions.append(nuc_mask)           
        return predictions



    def pred_cells(self, images, prot, image_data_list, precombined=False):
        """Predict the cell segmentation for a list of images.
        Keyword arguments:
        images -- list of lists of image paths/arrays. It should following the
                  pattern if with er channel input,
                  [
                      [microtubule_path0/image_array0, microtubule_path1/image_array1, ...],
                      [er_path0/image_array0, er_path1/image_array1, ...],
                      [nuclei_path0/image_array0, nuclei_path1/image_array1, ...]
                  ]
                  or if without er input,
                  [
                      [microtubule_path0/image_array0, microtubule_path1/image_array1, ...],
                      None,
                      [nuclei_path0/image_array0, nuclei_path1/image_array1, ...]
                  ]
                  The ER channel is required when multichannel is True
                  and required to be None when multichannel is False.
                  The images needs to be of the same size.
        precombined -- If precombined is True, the list of images is instead supposed to be
                       a list of RGB numpy arrays (default: False).
        Returns:
        predictions -- a list of predictions of cell segmentations.
        """

        def _preprocess(image):
            self.target_shape = image.shape
            if not len(image.shape) == 3:
                raise ValueError("image should have 3 channels")
            if not self.scale_factor == 1:
                cell_image = transform.rescale(image, self.scale_factor, multichannel=True)
            else:
                cell_image = image

            if self.padding:
                rows, cols = cell_image.shape[:2]
                self.scaled_shape = rows, cols
                cell_image = cv2.copyMakeBorder(
                    cell_image,
                    32,
                    (32 - rows % 32),
                    32,
                    (32 - cols % 32),
                    cv2.BORDER_REFLECT,
                )
            cell_image = cell_image.transpose([2, 0, 1])
            return cell_image

        if not precombined:
            images = self._image_conversion(images)
        preprocessed_imgs = list(map(_preprocess, images))
        predictions=[]
  
        desc = LocalBinaryPatterns(24, 8)
        if train_or_test == 'train':
            # make binary_patterns descriptor

            model = LinearSVC(C=100.0, random_state=42, max_iter=10000)
            model_data_list=[]
            model_label_list=[]
        else:
            ROOT = '../input/hpa-single-cell-image-classification/'
            model = load('../input/141617model/celltexture.joblib')
                
        #for i in range(len(preprocessed_imgs)):

        for i in tqdm(range(len(preprocessed_imgs))):
            mt,er,nu = preprocessed_imgs[i]
            pr=prot[i]
            image_data = image_data_list[i]
            im =image_data['image_id']
            label=image_data['train_label']
            
            
            maxval, minval, sumval, maxfreq, mostfreq, medianval = imageLevelStats(nu, image_data)
            image_data['nu_maxval']=maxval
            image_data['nu_minval']=minval
            image_data['nu_sumval']=sumval
            image_data['nu_maxfreq']=maxfreq
            image_data['nu_mostfreq']=mostfreq
            image_data['nu_medianval']=medianval
            
            maxval, minval, sumval, maxfreq, mostfreq, medianval = imageLevelStats(pr, image_data)
            image_data['pr_maxval']=maxval
            image_data['pr_minval']=minval
            image_data['pr_sumval']=sumval
            image_data['pr_maxfreq']=maxfreq
            image_data['pr_mostfreq']=mostfreq
            image_data['pr_medianval']=medianval
            
            maxval, minval, sumval, maxfreq, mostfreq, medianval = imageLevelStats(er, image_data)
            image_data['er_maxval']=maxval
            image_data['er_minval']=minval
            image_data['er_sumval']=sumval
            image_data['er_maxfreq']=maxfreq
            image_data['er_mostfreq']=mostfreq
            image_data['er_medianval']=medianval            
            
            maxval, minval, sumval, maxfreq, mostfreq, medianval = imageLevelStats(mt, image_data)
            image_data['mt_maxval']=maxval
            image_data['mt_minval']=minval
            image_data['mt_sumval']=sumval
            image_data['mt_maxfreq']=maxfreq
            image_data['mt_mostfreq']=mostfreq
            image_data['mt_medianval']=medianval        
            

            nuc_cont, nuc_midpoint, nuc_mask, nuc_rect = find_nuc_contours (nu)
            
            cell_graph = generateLayoutGraph(image_data, nuc_cont, nuc_midpoint, nuc_rect)
            contours = find_cells( mt, er, nu, nuc_cont, nuc_mask,nuc_midpoint,cell_graph)

            box_sample_list = compassSampleBoxes(image_data, nuc_cont, contours, nuc_mask, nuc_midpoint, pr)
            
            if train_or_test == 'train':
            # make binary_patterns descriptor
                addLbpData( desc, image_data, box_sample_list, pr, model_data_list, model_label_list)
            else:
                predictBoxlist(desc, model, image_data, box_sample_list, pr )
            
            #print('cell nodes', cell_graph.nodes)
            #print('cell edges', cell_graph.edges)
            #print(f'len(nuc_cont),{len(nuc_cont)}')
            for j in range(len(contours)):
                cont = contours[j]
                nucnt = nuc_cont[j]
                x,y,w,h = cv2.boundingRect(cont)
                cell = newCellDict(im, label, x, y, w, h)
                cell['nuc_bbox'] = cv2.boundingRect(nucnt)
                image_data['cells'].append(cell)
                cell['cellnum']=j
                cell['nuc_midpoint']=nuc_midpoint[j]


            analyseMicrotubules(image_data,contours,label,im,mt,pr,nuc_mask)
            analyseMicrotubulesFullCell(image_data,contours,label,im,mt,pr,nuc_mask)
            analyseER(image_data,contours,label,im,er,pr,nuc_mask)
            analyseNU(image_data,contours,label,im,nu,pr, nuc_mask)
            analyseMTNU(image_data,nuc_cont,label,im,nu,mt, nuc_mask)
            analyseNucleus(image_data,label,im, nu, er, mt, pr, nuc_cont, nuc_midpoint, nuc_mask, nuc_rect)
            #print(f'len(nuc_cont),{len(nuc_cont)}')
            analyseNucleusInnerRims(image_data, nuc_cont, nuc_midpoint, nu, pr)
            analyseNucleusOuterRims(image_data, nuc_cont, contours, nuc_midpoint, nu, pr)
            lookForAggresomes(image_data, pr)
            lookForNuclearBodies(image_data, pr, nuc_mask)
            lookForNuclearSpeckles(image_data, pr, nuc_mask)
            
            #if label == '2':
            if  debug and debug_show_input_images:
                printhist(nu)
                printhist(pr)
                printhist(er)
                printhist(mt)
                displaycopy= np.copy(nu)
                erdisplaycopy= np.copy(er)
                mtdisplaycopy= np.copy(mt)
                cv2.drawContours(displaycopy, nuc_cont,-1,255,1)
                cv2.drawContours(displaycopy, contours,-1,255,1)
                cv2.drawContours(pr, nuc_cont,-1,255,1)
                cv2.drawContours(pr, contours,-1,255,1)
                cv2.drawContours(erdisplaycopy, nuc_cont,-1,255,1)
                cv2.drawContours(erdisplaycopy, contours,-1,255,1)             
                cv2.drawContours(mtdisplaycopy, contours,-1,255,1)
                cv2.drawContours(mtdisplaycopy, nuc_cont,-1,255,1)
                print("Nucleus")
                do_display_image_big(displaycopy,image_data)
                print("Protein")
                do_display_image_big(pr,image_data)
                print("Endoplasmic reticulum")
                do_display_image_big(erdisplaycopy,image_data)
                print("Microtubules")
                do_display_image_big(mtdisplaycopy,image_data)




            calculateScores(image_data)



        return 


In [ ]:
def writeSubmissionFile (i, image_data_list, masks):
    if i == 0:
        f = open('submission.csv', 'w')
    else:
        f = open('submission.csv', 'a')
    with f as outf:
        if i == 0:
            print('ID,ImageWidth,ImageHeight,PredictionString', file=outf)        
        for image_data, cellmasklist in tqdm(zip(image_data_list,masks)):      
            image_id = image_data['image_id']
            w = image_data['width']
            h = image_data['height']            
            pred_strs=''
            c = 0

            for rle, cell  in zip(cellmasklist, image_data['cells']):
                if False and debug:
                    c = c + 1
                    rle = f'cell({i},{c})'
                if len(cell['cell_results']) == 0:
                    class_id = 18
                    cnf = 0.0001 
                    pred_strs += (f' {class_id} {cnf} {rle}')
                else:
                    for cell_result in cell['cell_results']:
                        class_id, cnf = cell_result
                        cnf = round(cnf,4)                    
                        pred_strs += (f' {class_id} {cnf} {rle}')                
            print(f'{image_id},{w},{h},{pred_strs}', file=outf)

In [ ]:
def outputImageData(image_data_list):
    image_field_excludes=['cells']
    cell_field_excludes=['nucleoli']
    nuc_field_excludes=[]
    
    if train_or_test == 'train' or debug:
        with open('cellstats.txt', 'a') as outf:
            i = newImageDict('', '',0,0)
            c = newCellDict('', '',0,0,0,0)
            n = newNucleolusDict('', '')
    
            header = '' 
            for field in i:
                if field not in image_field_excludes:
                    header += f'Img {field};'
            for field in c:
                if field not in cell_field_excludes:
                    header += f'Cel {field};'
            for field in n:
                if field not in nuc_field_excludes:
                    header += f'Nuc {field};'
            print(header, file = outf)
            
            for image in image_data_list:
                for cell in image['cells']:
                    for nuc in cell['nucleoli']:
                        row = ''
                        for field in i:
                            if field not in image_field_excludes:
                                row += f'{image[field]};'                    
                        for field in c:
                            if field not in cell_field_excludes:
                                row += f'{cell[field]};' 
                        for field in n:
                            if field not in nuc_field_excludes:
                                row += f'{nuc[field]};'
                        print(row, file = outf)

In [ ]:

cellsegmentor = CellSegmentator()
cellsegmentorOpenCV = CellSegmentatorOpenCV()

data_size = len(data_df)

bs = 150

def load_images(df : pd.DataFrame, root=f'../input/hpa-single-cell-image-classification/{train_or_test}/', resize = True):
    gray = []
    ryb = []
    protein=[]
    gray_orig = []
    ryb_orig = []
    protein_orig=[]
    image_data_list = []
    imWidth = []
    imHeight = []  
    
    if train_or_test == 'train' or train_or_test == 'traintest':
        root=f'../input/hpa-single-cell-image-classification/train/'
    else:
        root=f'../input/hpa-single-cell-image-classification/test/'
    
    for i, row in tqdm(df.iterrows(), total=len(df)):
        r = os.path.join(root, f'{row.ID}_red.png')
        y = os.path.join(root, f'{row.ID}_yellow.png')
        b = os.path.join(root, f'{row.ID}_blue.png')
        g = os.path.join(root, f'{row.ID}_green.png')
        r = cv2.imread(r, 0)
        y = cv2.imread(y, 0)
        b = cv2.imread(b, 0)
        g = cv2.imread(g, 0)

        
        #do_display_image_big(b)
        #do_display_image_big(g)
        #h1 = np.histogram(g, bins=25)
        #print(h1)
        #g[g>100]=0
        #h2 = np.histogram(g, bins=25)             
        #print(h2)
        #g2=g*2.5
        #do_display_image_big(g2)
        
        
        ryb_orig_image = np.stack((r, y, b), axis=2)
        if resize:
            gray_image = cv2.resize(b, (512, 512))

            ryb_image = cv2.resize(ryb_orig_image, (512, 512))
            gray.append(gray_image)
            ryb.append(ryb_image)
            protein_image =  cv2.resize(g, (512, 512))
            protein.append(g)
        #else:
        ryb_image_orig = np.stack((r, y, b), axis=2)
        gray_orig.append(b)
        ryb_orig.append(ryb_orig_image)    
        protein_orig.append(g)
            
        if train_or_test == 'train' or train_or_test == 'traintest':
            label = row.Label
        else:
            label = ''           

        ann = newImageDict(row.ID, label, b.shape[1],b.shape[0])

        image_data_list.append(ann)
    return gray, ryb, protein, gray_orig, ryb_orig, protein_orig,image_data_list

with open('cellstats.txt', 'w') as outf:
    print('', file = outf)
    
falsePos=np.zeros(19)
falseNeg=np.zeros(19)
truePos=np.zeros(19)
trueNeg=np.zeros(19)

print(data_df)

for i in range(0, data_size, bs):
    print('!!!!', i, '!!!!')
    start = i
    end = min(len(data_df), start + bs)
    batch_df = data_df[start:end]
    print(len(batch_df))
    print(f'---- start load images ----{start}, {end}')
    gray, ryb, pr, gray_orig, ryb_orig, pr_orig, image_data_list = load_images(batch_df,resize = True)


    idn=len(image_data_list)
    print( f'image_data_list {idn}')
    
    print(len(gray))
    print(f'---- finish load images --{start}, {end}')
    print(f'---- start pred nuclei --{start}, {end}')
    nuc_segmentations = cellsegmentor.pred_nuclei(gray)
    nsn=len(nuc_segmentations)
    print( f'nuc_segmentations{nsn}')    
    print(nuc_segmentations[0].shape)
    print(f'---- finish pred nuclei --{start}, {end}')
    print(f'---- start pred cells --{start}, {end}')
    cell_segmentations = cellsegmentor.pred_cells(ryb, precombined=True)
    
    nsn=len(nuc_segmentations)
    csn=len(cell_segmentations)
    print( f'nuc_segmentations{nsn} cell_segmentations {csn}')
    
    print(cell_segmentations[0].shape)
    print(f'---- finish pred cells --{start}, {end}')
    print(f'---- start pred cells OpenCV --{start}, {end}')
    cellsegmentorOpenCV.pred_cells(ryb_orig,pr_orig,image_data_list,precombined=True)
    print(f'---- finish pred cells --{start}, {end}')
    print(f'---- start calculating masks ----{i}')
    
    outputImageData(image_data_list)
    kernel = np.ones((5, 5), 'uint8') 
    maskRLElists=[]
    zcount=0
    nsn=len(nuc_segmentations)
    csn=len(cell_segmentations)
   
    idn=len(image_data_list)
    print( f'nuc_segmentations{nsn} cell_segmentations {csn} \
  image_data_list {idn}')
    for image_data, nuc_seg, cell_seg in tqdm (zip(image_data_list, nuc_segmentations, \
                                          cell_segmentations )):
        zcount= zcount+1
        maskRLElist = []
        maskRLElists.append(maskRLElist)
        ll=len(maskRLElists)
        imid=image_data['image_id']

        #print(f'{zcount}: {imid} mask list length {ll}')
        nuc_small, cell_small = label_cell(nuc_seg, cell_seg)
        
        #orig_size = gray_orig[i].shape[0]
        orig_size = image_data['width']
        cell=cv2.resize(cell_small,(orig_size,orig_size), cv2.INTER_NEAREST_EXACT)
        #print(image_data['image_id'])
        cell_data=image_data['cells']
        for cell_data in image_data['cells']:
            nuc_midpoint = cell_data['nuc_midpoint']
            mx,my=nuc_midpoint
            cell_mask_label=cell[my,mx]
            maskImage = np.zeros((orig_size,orig_size), np.uint8)
            maskImageBool = np.zeros((orig_size,orig_size), np.bool8)
            maskImage[cell==cell_mask_label]=255
            #do_display_image_big(maskImage)
            # erode to remove spurious effects from previous resize
            ed=cv2.erode(maskImage,kernel,iterations=2)

            maskImageBool[ed==255]=True
            #do_display_image_big(maskImageBool)
            rle = encode_binary_mask(maskImageBool)
            maskRLElist.append(rle)
   
     
    print(f'---- finish calculating masks ----{i}')
    print(f'---- start writing batch to file ----{i}')
    print('image_data length', len(image_data_list))
    #print('mask lists length', len(maskRLElists))
    writeSubmissionFile ( i, image_data_list, maskRLElists)
    print(f'---- finish writing batch to file ----{i}')

                
    if train_or_test == 'train' or train_or_test == 'traintest':
        countTrainingScores(image_data_list,falsePos,falseNeg,truePos,trueNeg)

    



In [ ]:
!wc -l submission.csv

In [ ]:
print(len(cell_data))